# <center> Прогнозирование рейтинга отелей booking.com

# Постановка задачи

**Проблема** компании Booking  — это нечестные отели, которые накручивают себе рейтинг.

**Цель проекта** - построение модели на основе машинного обучения, которая предсказывает рейтинг отеля.

**Задачи проекта:**
1. Ознакомится с входными данными (DataFrame с информацией об отелях);
2. Изучить пример машинного обучения (scikit - learn класс RandomForestRegressor)
3. Выполнить подготовку данных, которые будут использованы для обучения модели;
4. Проверить эффективность, предлагаемой модели, используя метрику MAPE;
5. Принять участие в соревнованиях на площадке kaggle.com

Соревнование kaggle с первоначальными данными находится по [ссылке](https://www.kaggle.com/competitions/sf-booking).

Файлы необходимые для обучения модели:
- hotels_train.csv - набор данных для обучения
- hotels_test.csv - набор данных для оценки качества
- submission.csv - файл сабмишна в нужном формате

**Признаки:**
- `hotel_address` - адрес отеля
- `review_date` - дата, когда рецензент разместил соответствующий отзыв.
- `average_score` - средний балл отеля, рассчитанный на основе последнего комментария за последний год
- `hotel_name` - название отеля
- `reviewer_nationality` - национальность рецензента
- `negative_review` - отрицательный отзыв, который рецензент дал отелю.
- `review_total_negative_word_counts` - общее количество слов в отрицательном отзыв
- `positive_review` - положительный отзыв, который рецензент дал отелю
- `review_total_positive_word_counts` - общее количество слов в положительном отзыве
- `reviewer_score` - оценка, которую рецензент поставил отелю на основе своего опыта
- `total_number_of_reviews_reviewer_has_given` - количество отзывов, которые рецензенты дали в прошлом
- `total_number_of_reviews` - общее количество действительных отзывов об отеле
- `tags` - теги, которые рецензент дал отелю.
- `days_since_review` - продолжительность между датой проверки и датой очистки
- `additional_number_of_scoring` - есть также некоторые гости, которые просто поставили оценку сервису, а не оставили отзыв. Это число указывает, сколько там действительных оценок без проверки.
- `lat` - широта отеля
- `lng` - долгота отеля

# Загрузка и предварительная обработка данных

### Импорт библиотек и данных

In [1]:
# Импорт библиотек
import numpy as np
import pandas as pd
import re
import time
import category_encoders as ce
from sklearn.feature_selection import f_classif, chi2
from scipy import stats
from scipy.stats import kstest
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import matthews_corrcoef
import gdown

# Для визуализации
import matplotlib.pyplot as plt
import seaborn as sns
# Для красоты
sns.set_theme('notebook', style="whitegrid", palette="Set2")

# Для логирования
from comet_ml import Experiment
from config import settings

# Для разделения датасета:
from sklearn.model_selection import train_test_split

# Для работы с координатами
from geopy.geocoders import Photon
from geopy.distance import geodesic

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
"""
# Создаю эксперимент
experiment = Experiment(
    api_key=settings['api_key'],
    project_name=settings['project_name'],
    workspace=settings['workspace'],
    auto_param_logging=False,
    display_summary_level=0
)
"""

"\n# Создаю эксперимент\nexperiment = Experiment(\n    api_key=settings['api_key'],\n    project_name=settings['project_name'],\n    workspace=settings['workspace'],\n    auto_param_logging=False,\n    display_summary_level=0\n)\n"

In [3]:
# Фиксируем RANDOM_SEED
RANDOM_SEED = 42

In [4]:
# Фиксируем версию пакетов:
# !pip freeze > requirements.txt

In [5]:
# Загружаем данные
# Раскомментировать для kaggle
# DATA_DIR = '/kaggle/input/sf-booking/'
# sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн
gdown.download("https://drive.google.com/uc?id=1R76T-wGJ1HKphWRFrRKwkNP6_gjWh4zt", "hotels_train.csv")
df_train = pd.read_csv("hotels_train.csv") # датасет для обучения
gdown.download("https://drive.google.com/uc?id=1tZmdSLpx_9DutzVdi__w4UFFH1N_x4uZ", "hotels_test.csv")  
df_test = pd.read_csv("hotels_test.csv") # датасет для предсказания

Downloading...
From (original): https://drive.google.com/uc?id=1R76T-wGJ1HKphWRFrRKwkNP6_gjWh4zt
From (redirected): https://drive.google.com/uc?id=1R76T-wGJ1HKphWRFrRKwkNP6_gjWh4zt&confirm=t&uuid=776b25c9-75db-4d5d-8f16-683cea70565f
To: c:\Users\milov\OneDrive\Рабочий стол\учеба\DS\Конспекты DS\Разведывательный анализ\PROJECT-3. EDA\hotels_train.csv
100%|██████████| 178M/178M [01:15<00:00, 2.35MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1tZmdSLpx_9DutzVdi__w4UFFH1N_x4uZ
To: c:\Users\milov\OneDrive\Рабочий стол\учеба\DS\Конспекты DS\Разведывательный анализ\PROJECT-3. EDA\hotels_test.csv
100%|██████████| 58.9M/58.9M [00:22<00:00, 2.63MB/s]


In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

In [7]:
df_train.head(2)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097


In [8]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128935 entries, 0 to 128934
Data columns (total 16 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               128935 non-null  object 
 1   additional_number_of_scoring                128935 non-null  int64  
 2   review_date                                 128935 non-null  object 
 3   average_score                               128935 non-null  float64
 4   hotel_name                                  128935 non-null  object 
 5   reviewer_nationality                        128935 non-null  object 
 6   negative_review                             128935 non-null  object 
 7   review_total_negative_word_counts           128935 non-null  int64  
 8   total_number_of_reviews                     128935 non-null  int64  
 9   positive_review                             128935 non-null  object 
 

In [9]:
df_test.head(2)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,tags,days_since_review,lat,lng
0,Via Senigallia 6 20161 Milan Italy,904,7/21/2017,8.1,Hotel Da Vinci,United Kingdom,Would have appreciated a shop in the hotel th...,52,16670,Hotel was great clean friendly staff free bre...,62,1,"[' Leisure trip ', ' Couple ', ' Double Room '...",13 days,45.533137,9.171102
1,Arlandaweg 10 Westpoort 1043 EW Amsterdam Neth...,612,12/12/2016,8.6,Urban Lodge Hotel,Belgium,No tissue paper box was present at the room,10,5018,No Positive,0,7,"[' Leisure trip ', ' Group ', ' Triple Room ',...",234 day,52.385649,4.834443


In [10]:
# sample_submission.head(2)

In [11]:
# sample_submission.info()

Данные загружены без ошибок

### Подготовка данных

In [12]:
# Проверим сразу дубликаты и удалим их если имеются

# Тренировочные данные - можно удалять дубликаты
print(f'В тренировочных данных {df_train[df_train.duplicated()].shape[0]} дубликатов')
df_train.drop_duplicates(inplace=True)
print('Количество строк после удаления дубликатов: {}'.format(df_train.shape[0]))

# Тестовые данные - нельзя удалять дубликаты, только проверяем
print(f'В тестовом DataFrame {df_test[df_test.duplicated()].shape[0]} дубликатов')
# df_test.drop_duplicates(inplace=True)

В тренировочных данных 307 дубликатов
Количество строк после удаления дубликатов: 386496
В тестовом DataFrame 29 дубликатов


In [13]:
# Для корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест

# В тесте у нас нет значения reviewer_score, мы его должны предсказать, 
# поэтому пока просто заполняем нулями
df_test['reviewer_score'] = 0

data = pd.concat([df_test, df_train], sort=False, ignore_index=True) # объединяем

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515431 entries, 0 to 515430
Data columns (total 18 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               515431 non-null  object 
 1   additional_number_of_scoring                515431 non-null  int64  
 2   review_date                                 515431 non-null  object 
 3   average_score                               515431 non-null  float64
 4   hotel_name                                  515431 non-null  object 
 5   reviewer_nationality                        515431 non-null  object 
 6   negative_review                             515431 non-null  object 
 7   review_total_negative_word_counts           515431 non-null  int64  
 8   total_number_of_reviews                     515431 non-null  int64  
 9   positive_review                             515431 non-null  object 
 

In [15]:
missing_data = data.isnull().sum()
missing_data[missing_data > 0]

lat    3268
lng    3268
dtype: int64

Пропущенные значения в признаках долготы и широты, попробуем получить их координаты с помощью адресов, используя библиотеку `geopy`

In [16]:
# Находим адреса, у которых отсутствуют координаты
missing_coords_mask = data['lat'].isna() & data['lng'].isna()
addresses_without_coords = data[missing_coords_mask]['hotel_address'].unique()

print(f"Адресов без координат: {len(addresses_without_coords)}")
addresses_without_coords = addresses_without_coords.tolist()

Адресов без координат: 17


In [17]:
# Инициализация геокодера Photon
# Добавила задержки, были случаи когда выдавал ошибку код
geolocator = Photon(user_agent="hotel_reviews_analysis", timeout=2)

for address in addresses_without_coords:
    location = geolocator.geocode(address)
    if location:
        mask = data['hotel_address'] == address
        data.loc[mask, 'lat'] = location.latitude
        data.loc[mask, 'lng'] = location.longitude
    time.sleep(2)

In [18]:
missing_data = data.isnull().sum()
missing_data

hotel_address                                 0
additional_number_of_scoring                  0
review_date                                   0
average_score                                 0
hotel_name                                    0
reviewer_nationality                          0
negative_review                               0
review_total_negative_word_counts             0
total_number_of_reviews                       0
positive_review                               0
review_total_positive_word_counts             0
total_number_of_reviews_reviewer_has_given    0
tags                                          0
days_since_review                             0
lat                                           0
lng                                           0
sample                                        0
reviewer_score                                0
dtype: int64

### Очистка данных

Пропущенные данные заполнены, теперь проверим какие данные содержаться в object признаках, для понимания необходима ли для них очистка или преобразования в другой формат

In [19]:
# Выбор столбцов типа object
object_columns = data.select_dtypes(include=['object']).columns

# Вывод по одному примеру из каждого object-столбца
for col in object_columns:
    print(f'Столбец: {col}')
    print(f'Пример данных: "{data[col].iloc[0]}"')
    print(f"Тип данных: {type(data[col].iloc[0])}")
    print('-' * 50)

Столбец: hotel_address
Пример данных: "Via Senigallia 6 20161 Milan Italy"
Тип данных: <class 'str'>
--------------------------------------------------
Столбец: review_date
Пример данных: "7/21/2017"
Тип данных: <class 'str'>
--------------------------------------------------
Столбец: hotel_name
Пример данных: "Hotel Da Vinci"
Тип данных: <class 'str'>
--------------------------------------------------
Столбец: reviewer_nationality
Пример данных: " United Kingdom "
Тип данных: <class 'str'>
--------------------------------------------------
Столбец: negative_review
Пример данных: " Would have appreciated a shop in the hotel that sold drinking water etc but not necessity Would recommend if like us you arrive late at night to bring drinks from plane airport as there s no shop nearby There is a minibar though if you want to pay those prices "
Тип данных: <class 'str'>
--------------------------------------------------
Столбец: positive_review
Пример данных: " Hotel was great clean friendl

Судя по первоначальным данным, как минимум один числовой признак - количество дней с последнего отзыва имеет не верный формат, преобразуем сразу в числовой. Кроме того признак даты представлен в object формате, преобразуем и его. Заодно признак sample переведем в категориальный тип данных, чтобы не занимал лишней памяти. Признаки отзывов, тегов и национальности, имеют лишние пробелы в данных, у тегов есть лишние символы. Исправим это:

In [20]:
# Преобразование days_since_review в числовой формат
data['days_since_review'] = (
    data['days_since_review']
    .str.replace(' days', '')
    .str.replace(' day', '').astype('int16')
)
# Уменьшение веса sample
data['sample'] = data['sample'].astype('category')
# review_date признак даты, преобразуем в datetime
data['review_date'] = pd.to_datetime(data['review_date'])

# Очищаю reviewer_nationality от лишних пробелов
data['reviewer_nationality']=data['reviewer_nationality'].str.strip()

# Очищаю отзывы от лишних пробелов, перевожу все в нижний регистр для удобства работы
data['negative_review'] = data['negative_review'].str.strip().str.lower()
data['positive_review'] = data['positive_review'].str.strip().str.lower()

# Из тегов уберем лишние пробелы и скобки
data['tags'] = (
    data['tags'].str.strip("[]'").str.replace(" ', ' ", ", ").str.strip()
)
# Смотрим, что получилось
for col in object_columns:
    print(f'Столбец: {col}')
    print(f'Пример данных: "{data[col].iloc[0]}"')
    print(f"Тип данных: {type(data[col].iloc[0])}")
    print('-' * 50)

Столбец: hotel_address
Пример данных: "Via Senigallia 6 20161 Milan Italy"
Тип данных: <class 'str'>
--------------------------------------------------
Столбец: review_date
Пример данных: "2017-07-21 00:00:00"
Тип данных: <class 'pandas._libs.tslibs.timestamps.Timestamp'>
--------------------------------------------------
Столбец: hotel_name
Пример данных: "Hotel Da Vinci"
Тип данных: <class 'str'>
--------------------------------------------------
Столбец: reviewer_nationality
Пример данных: "United Kingdom"
Тип данных: <class 'str'>
--------------------------------------------------
Столбец: negative_review
Пример данных: "would have appreciated a shop in the hotel that sold drinking water etc but not necessity would recommend if like us you arrive late at night to bring drinks from plane airport as there s no shop nearby there is a minibar though if you want to pay those prices"
Тип данных: <class 'str'>
--------------------------------------------------
Столбец: positive_review
При

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515431 entries, 0 to 515430
Data columns (total 18 columns):
 #   Column                                      Non-Null Count   Dtype         
---  ------                                      --------------   -----         
 0   hotel_address                               515431 non-null  object        
 1   additional_number_of_scoring                515431 non-null  int64         
 2   review_date                                 515431 non-null  datetime64[ns]
 3   average_score                               515431 non-null  float64       
 4   hotel_name                                  515431 non-null  object        
 5   reviewer_nationality                        515431 non-null  object        
 6   negative_review                             515431 non-null  object        
 7   review_total_negative_word_counts           515431 non-null  int64         
 8   total_number_of_reviews                     515431 non-null  int64        

При предварительной обработке данных были обнаружены и удалены дубликаты в тренировочной выборке (тестовую не трогали, во избежание будущих ошибок на Kaggle). Так же, были обнаружены пропущенные значения в координатах. При анализе адресов, где пропущены координаты, обнаружены ошибки, исправлять данные ошибки смысла не вижу, так как полный адрес нам не понадобится. Координаты заполнены с помощью геокодера Photon из библиотеки geopy. Геокодер смог получить координаты не смотря на ошибки, пропущенных значений больше нет. Так же были обнаружены ошибки в значениях, которые успешно исправлены. Можно переходить к следующему этапу исследования. 

### Функции необходимые далее в работе

In [22]:
# Переменные
alpha = 0.05

# Сразу задаю маску для получения только трейн данных, где рейтинг не 0
train_mask = data['sample'] == 1
train_data = data[train_mask].copy()


# Определяю необходимые мне функции
def get_stats(groupby_column, df = None, stat_column='reviewer_score'):
    """Функция, выводящая статистику указанного параметра по группам.
    
    Args:
        df (DataFrame): DataFrame с данными, по умолчанию data[train_mask]
        groupby_column (array-like): столбец для группировки
        stat_column (str): столбец для анализа, по умолчанию 'reviewer_score'
        
    Returns:
        DataFrame: статистика параметра по группам
    """
    if df is None:
        df = data[train_mask]
    stats = (
        df.groupby(groupby_column, observed=False)[stat_column]
        .describe()
        .round(2)
    )
    print(f'Статистика {stat_column} по {groupby_column}:')
    display(stats)

def get_boxplots(
    columns, titles=None, xlabels=None, df=None,
    ncols=3, colors=False, figsize=None):
    """Построение нескольких боксплотов в одном ряду
    
    Args:
        columns (list): Список колонок для графиков
        titles (list): Список заголовков. По умолчанию None
        xlabels (list): Список подписей оси X. По умолчанию None
        df (DataFrame): Данные. По умолчанию data[train_mask]
        ncols (int): Количество колонок. По умолчанию 3
        colors (bool): Поменять порядок цветов в палитре. По умолчанию False
        figsize (tuple): Размер figure (width, height). По умолчанию None
        
    Returns:
        Figure: Объект figure для дальнейших операций
    """
    df = data[train_mask] if df is None else df
    titles = columns if titles is None else titles
    xlabels = [''] * len(columns) if xlabels is None else xlabels
    
    nrows = (len(columns) + ncols - 1) // ncols
    
    # Если размер не задан
    if figsize is None:
        # Определяем высоту в зависимости от наличия подписей
        if all(label == '' for label in xlabels):  # если все подписи пустые
            figsize = (5*ncols, 4*nrows)
        else:
            figsize = (5*ncols, 5*nrows)
    
    # Создаем figure и axes
    fig, axes = plt.subplots(nrows, ncols, figsize=figsize)
    # Создаем одномерный массив, если графиков больше 1
    axes = [axes] if nrows == 1 and ncols == 1 else axes.ravel()
    for i, (col, title, xlabel) in enumerate(zip(columns, titles, xlabels)):
        # Настраиваем палитру
        palette = (
            sns.color_palette("Set2")[1::-1]
            if colors and df[col].nunique() == 2 else None
        )
        if i < len(axes):
            sns.boxplot(data=df, x=col, y='reviewer_score', hue=col, legend=False,
                        fliersize=2, linewidth=1.2, ax=axes[i], palette=palette)
            axes[i].set_title(title)
            axes[i].set_xlabel(xlabel)
            axes[i].set_ylabel('Рейтинг' if i % ncols == 0 else '')
    
    # Скрываем лишние
    for i in range(len(columns), nrows*ncols):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    return fig

def binary_test_norm(features_list, df = None, target='reviewer_score'):
    """Функция, выводящая сообщение о нормальности распределения бинарных признаков.
    
    Args:
        features_list (list): список проверяемых бинарных признаков
        df (DataFrame): DataFrame с данными, по умолчанию data[train_mask]
        target (str): столбец для анализа, по умолчанию 'reviewer_score'
        
    Returns:
        None: выводит сообщения о нормальности распределения категорий в признаках
    """
    if df is None:
        df = data[train_mask]
    normal_features = []
    for col in features_list:
        # Берем данные для двух групп
        group_1 = df[df[col] == 1][target]
        group_0 = df[df[col] == 0][target]
        
        # Проверяем группу 1
        normalized_1 = (group_1 - group_1.mean()) / group_1.std()
        _, p_value_1 = kstest(normalized_1, 'norm')
        if p_value_1 > alpha:
            normal_features.append(f"{col}=1")
            print(f'{col}=1: нормально распределены')
        
        # Проверяем группу 0
        normalized_0 = (group_0 - group_0.mean()) / group_0.std()
        _, p_value_0 = kstest(normalized_0, 'norm')
        if p_value_0 > alpha:
            normal_features.append(f"{col}=0")
            print(f'{col}=0: нормально распределены')
    
    # Итоговый вывод
    if len(normal_features) == 0:
        print("Нормально распределенных данных нет")


def categorical_test_norm(features_list, df = None, target='reviewer_score'):
    """Функция, выводящая сообщение о нормальности распределения категориальных
    признаков.
    
    Args:
        features_list (list): список проверяемых категориальных признаков
        df (DataFrame): DataFrame с данными, по умолчанию data[train_mask]
        target (str): столбец для анализа, по умолчанию 'reviewer_score'
        
    Returns:
        None: выводит сообщения о нормальности распределения категорий в признаках
    """
    if df is None:
        df = data[train_mask]
    normal_features = []
    for col in features_list:
        # Для каждой категории в признаке
        for category in df[col].unique():
            group_data = df[df[col] == category][target]
            normalized_data = (group_data - group_data.mean()) / group_data.std()
            _, p_value = kstest(normalized_data, 'norm')
            
            if p_value > alpha:
                normal_features.append(f"{col}={category}")
                print(f'{col}={category}: нормально распределены')
    # Итоговый вывод
    if len(normal_features) == 0:
        print("Нормально распределенных данных нет")

def recommend_test(n_groups, is_dependent, is_normal):
    """Рекомендует статистический тест на основе характеристик данных.
    
    Args:
        n_groups (int): количество групп для сравнения (1, 2, 3+)
        is_dependent (int): 1 если группы зависимые, 0 если независимые
        is_normal (int): 1 если распределение нормальное, 0 если нет
        
    Returns:
        str: рекомендация по статистическому тесту
        
    """
    # Проверка валидности входных данных
    if is_dependent not in [0, 1] or is_normal not in [0, 1]:
        return "Ошибка: некорректные входные параметры"
    
    # Логика выбора теста
    if n_groups == 1:
        if is_normal:
            test = "Одновыборочный t-критерий: scipy.stats.ttest_1samp"
        else:
            test = ("Критерий знаков (для одной выборки): "
                    "statsmodels.stats.descriptivestats.sign_test")
        
    elif n_groups == 2:
        if is_dependent:
            if is_normal:
                test = "Парный t-критерий: scipy.stats.ttest_rel"
            else:
                test = "Критерий Уилкоксона: scipy.stats.wilcoxon"
        else:
            if is_normal:
                test = ("Двухвыборочный t-критерий: scipy.stats.ttest_ind. "
                        "Надо проверить равенство дисперсий")
            else:
                test = "U-критерий Манна-Уитни: scipy.stats.mannwhitneyu"
            
    else:  # n_groups >= 3
        if is_dependent:
            if is_normal:
                test = "ANOVA с повторными измерениями: statsmodels.stats.anova.AnovaRM"
            else:
                test = "Критерий Фридмана: scipy.stats.friedmanchisquare"
        else:
            if is_normal:
                test = ("ANOVA: scipy.stats.f_oneway. "
                        "Проверь дисперсию, если не равна используй is_normal=0")
            else:
                test = "Критерий Краскела-Уоллиса: scipy.stats.kruskal"
    
    return f"Рекомендуемый тест: {test}"

def pairwise_u_test(features_list, is_binary=False, df=None, target='reviewer_score', 
                   alpha=0.05, bonferroni=False):
    """Функция, проводящая тест U-критерий Манна-Уитни сразу для нескольких признаков,
    или категорий из признака
    
    Args:
        features_list (list): список проверяемых категориальных или бинарных признаков
        is_binary (bool): флаг, если True, проверяется бинарный признак.
            По умолчанию False - проверяются категории в категориальном признаке
        df (DataFrame): DataFrame с данными, по умолчанию data[train_mask]
        target (str): столбец для анализа, по умолчанию 'reviewer_score'
        alpha (float): уровень значимости, по умолчанию 0.05
        bonferroni (bool): применять поправку Бонферрони, по умолчанию False
        
    Returns:
        None: выводит сообщения отвечающие на вопрос можем ли мы опровергнуть нулевую
        гипотезу
    """
    if df is None:
        df = data[train_mask]
    no_difference_features = []
    for col in features_list:
        if is_binary:
            categories = [1, 0]
            cat_names = {1: '1', 0: '0'}
        else:
            categories = df[col].unique()
            cat_names = {cat: str(cat) for cat in categories}
            n_pare = len(categories) * (len(categories) - 1) // 2
        # Определяем уровень значимости с поправкой
        current_alpha = alpha / n_pare if bonferroni else alpha    
        has_no_difference = False
        # Проверяем все возможные пары
        for i in range(len(categories)):
            for j in range(i + 1, len(categories)):
                cat1 = categories[i]
                cat2 = categories[j]
                
                group_1 = df[df[col] == cat1][target]
                group_2 = df[df[col] == cat2][target]
                
                _, p = stats.mannwhitneyu(group_1, group_2)
                
                if p > current_alpha:
                    if not has_no_difference:
                        print(f'\n{col}')
                        has_no_difference = True
                    print(f'{cat_names[cat1]} и {cat_names[cat2]}: '
                          f'p-value={p:.3f} > {current_alpha:.3f}. '
                          f'У нас нет оснований отвергнуть нулевую гипотезу.')
        if has_no_difference:
            no_difference_features.append(col)
        else:
            print(f'\n{col}: Отвергаем нулевую гипотезу')
    # Если все признаки имеют различая
    if not no_difference_features:
        print("Во всех признаках все группы статистически различаются")


def kruskal_test(features_list, df=None, target='reviewer_score'):
    """Функция, проводящая тест Крускала-Уоллиса сразу для нескольких признаков, .
    
    Args:
        features_list (list): список проверяемых категориальных признаков
        df (DataFrame): DataFrame с данными, по умолчанию data[train_mask]
        target (str): столбец для анализа, по умолчанию 'reviewer_score'
        
    Returns:
        None: выводит сообщения о результатах проверки нулевой гипотезы
    """
    if df is None:
        df = data[train_mask]
    
    no_difference_features = []
    
    for col in features_list:
        # Собираем все группы для признака
        groups = []
        for category in df[col].unique():
            group_data = df[df[col] == category][target]
            groups.append(group_data)
        
        # Проверяем критерием Крускала-Уоллиса
        _, p = stats.kruskal(*groups)
        
        if p > alpha:
            print(f'Для признака {col}: p-value:{p:.3f}>{alpha}. '
                  f'У нас нет оснований отвергнуть нулевую гипотезу.')
            no_difference_features.append(col)  # ← исправлено с feature на col
        else:
            print(f'Для признака {col}: отвергаем нулевую гипотезу '
                  f'(p-value:{p:.3f}<={alpha})')
    
    if not no_difference_features:
        print("Отвергаем нулевую гипотезу для всех признаков")

# Разведывательный анализ данных

## Предварительный анализ признаков

### Числовые признаки

In [23]:
# Выведем на экран основные статические данные
data[train_mask].describe(include='number')

,additional_number_of_scoring,average_score,review_total_negative_word_counts,total_number_of_reviews,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,days_since_review,lat,lng,reviewer_score
count,386496.000000,386496.000000,386496.000000,386496.000000,386496.000000,386496.000000,386496.000000,386496.000000,386496.000000,386496.000000
mean,498.504375,8.397421,18.539400,2744.679231,17.778163,7.176211,354.339339,49.433914,2.892783,8.397299
std,500.365093,0.547861,29.703569,2316.934876,21.724766,11.052453,208.958145,3.462264,4.667918,1.635747
min,1.000000,5.200000,0.000000,43.000000,0.000000,1.000000,0.000000,41.328376,-0.369758,2.500000
25%,169.000000,8.100000,2.000000,1161.000000,5.000000,1.000000,175.000000,48.214277,-0.143372,7.500000
50%,342.000000,8.400000,9.000000,2134.000000,11.000000,3.000000,353.000000,51.499981,0.019886,8.800000
75%,660.000000,8.800000,23.000000,3633.000000,22.000000,8.000000,527.000000,51.516288,4.841163,9.600000
max,2682.000000,9.800000,408.000000,16670.000000,395.000000,355.000000,730.000000,52.400181,16.429233,10.000000


In [24]:
# Получаем числовые колонки, исключаем sample и координаты
exclude_cols = ['sample', 'lat', 'lng']
num_cols = [col for col in data.select_dtypes(include=['number']).columns 
           if col not in exclude_cols]

# Посмотрим на пример данных на одном случайном отзыве
data[train_mask][['hotel_name', *num_cols]].sample(1).iloc[0]

hotel_name                                    The Rembrandt
additional_number_of_scoring                            421
average_score                                           8.5
review_total_negative_word_counts                        53
total_number_of_reviews                                1802
review_total_positive_word_counts                        75
total_number_of_reviews_reviewer_has_given                4
days_since_review                                       183
reviewer_score                                          9.6
Name: 361062, dtype: object

In [25]:
# Посмотрим на распределение основных числовых признаков
# Гистограммы
data[train_mask][num_cols].hist(bins=30, figsize=(15, 10))
plt.suptitle('Гистограммы числовых признаков')
plt.tight_layout()
# Раскомментировать для отображения
# plt.show()
plt.close()

<div align="center">
  <img src="data/image/output.png">
</div>

Целевая переменная `reviewer_score` имеет значение в диапазоне от 2,5 до 10 баллов, причем явно выраженный пик распределения показывает именно на 10 баллах. Не понятна градация оценивания, вряд ли человек ставит оценку, как в тестовом примере 7,1 балла, видимо балл рассчитывается системой на основе отзыва/тегов. Средний балл отеля `average_score` в отличие от рейтинга в диапазоне от 5 до 10 баллов, тем не менее распределение напоминает нормальное с пиками в области средних значений. Медианы и средние данных признаков почти не отличаются, необходимо проверить корреляцию между ними позже.

`days_since_review` - почти равномерное распределение, в выборке имеются как совсем новые отзывы, так и двухлетней давности. Вряд ли данный признак сильно влияет на целевую переменную, однако возможно более новые отзывы более достоверно указывают на рейтинг, чем старые.

`total_number_of_reviews_reviewer_has_given` - 75% людей оставили не более 8 отзывов, В среднем один пользователь booking оставляет не более 3 отзывов, но имеются сильные выбросы (максимальное количество отзывов - 355). Возможно это и есть искомые накрученные отзывы.

`review_total_positive_word_counts`, `review_total_negative_word_counts` признаки имеет значения от 0 и до примерно 400, что говорит о том, что в выборке имеются разного рода отзывы: отзывы, где указана только оценка, и более развернутые детальные отзывы. В среднем люди в отзывы указывают не более 20 слов (т.е. пара предложений).

`total_number_of_reviews` - имеются отели с очень большим количеством отзывов, по всей видимости они очень популярны, в целом об одном отеле в среднем оставлено около 2000 отзывов.

`additional_number_of_scoring` - я не очень понимаю данный признак, взаимодействует ли он с общим количеством отзывов по отелю. Если это число отзывов, где нет отзыва а есть только оценка, то стоит проверить соотношение additional/total, то есть сколько молчаливых оценок поставлено отелю, возможно данный показатель укажет прямо на отели, завышающие себе оценку искусственно. Данный признак лежит в диапазоне от 1 до более чем 2000, что довольно большая цифра. Однако возможно признак коррелирует с - `total_number_of_reviews`, их распределения очень похожи и 2000 молчаливых оценок, это оценки самого популярного отеля, где оставлено почти 17 тысяч отзывов.

In [26]:
# Теперь зная значения числовых столбцов, преобразую тип данных, для снижения "веса"
cols_to_int16 = [
    'review_total_negative_word_counts', 'review_total_positive_word_counts', 
    'total_number_of_reviews_reviewer_has_given', 'additional_number_of_scoring'
]
for col in cols_to_int16:
    data[col] = data[col].astype('int16')

cols_to_float32 = ['average_score', 'reviewer_score', 'lat', 'lng']
for col in cols_to_float32:
    data[col] = data[col].astype('float32')
    
data['total_number_of_reviews'] = data['total_number_of_reviews'].astype('int32')

### Не числовые признаки

In [27]:
# Посмотрим на пример данных на одном случайном отзыве
cat_cols = [col for col in data.select_dtypes(include=['object']).columns]
data[train_mask][['hotel_name', *cat_cols]].sample(1).iloc[0]

hotel_name                                      Catalonia Barcelona Plaza
hotel_address           Plaza Espa a 6 8 Sants Montju c 08014 Barcelon...
hotel_name                                      Catalonia Barcelona Plaza
reviewer_nationality                                                Italy
negative_review                                               no negative
positive_review         i have received a very welcome surprise by off...
tags                    Business trip, Solo traveler, Double or Twin R...
Name: 395267, dtype: object

In [28]:
# Основные статистические характеристики категориальных данных
data.describe(include='object')

,hotel_address,hotel_name,reviewer_nationality,negative_review,positive_review,tags
count,515431,515431,515431,515431,515431,515431
unique,1493,1492,227,323166,401607,55242
top,163 Marsh Wall Docklands Tower Hamlets London ...,Britannia International Hotel Canary Wharf,United Kingdom,no negative,no positive,"Leisure trip, Couple, Double Room, Stayed 1 ni..."
freq,4789,4789,245165,127816,35924,5101


In [29]:
# Основные статистические характеристики review_date
data['review_date'].describe()

count                           515431
mean     2016-08-13 14:16:05.515849728
min                2015-08-04 00:00:00
25%                2016-02-23 00:00:00
50%                2016-08-15 00:00:00
75%                2017-02-09 00:00:00
max                2017-08-03 00:00:00
Name: review_date, dtype: object

1492 уникальных значения для `hotel_name`, тогда как для `hotel_address` 1493 уникальных значения. Видимо имеются отели с одинаковым названием. Большая часть отзывов (почти половина) опубликована гражданами Англии, всего в выборке представлено 227 различных уникальных национальности (`reviewer_nationality`). Самый частые негативный отзыв (`negative_review`) не негативный (почти 30% данных - 'no negative'). Для положительных отзывов (`positive_review`) картина такая же, самый частый отзыв не позитивный ('no positive'), но уже куда меньшее количество, видимо люди склонны больше к положительным отзывам, чем отрицательным. Количество уникальных тегов ни о чем на не говорит, так как в данных хранится список тегов, однако уже видно, что из него можно спроектировать много новых признаков, таких как количество дней поездки, тип комнаты, тип поездки и тп. Данные в выборке представлены за два года, с августа 2015 по август 2017 года (`review_date`).

## Проектирование признаков

На основе предварительного анализа сформулированы следующие гипотезы для проверки:

Даты - `review_date`, `days_since_review`:
- Год отзыва влияет на рейтинг отеля
- Месяц отзыва влияет на рейтинг отеля
- Квартал отзыва влияет на рейтинг отеля
- Свежесть отзыва оказывает влияние на рейтинг отеля

Локация - `hotel_address`, `hotel_name`, `lat`, `lng`:
- Страна расположения отеля влияет на рейтинг отеля
- Город расположения отеля влияет на рейтинг отеля
- Расстояние до центра города оказывает влияние на рейтинг отеля
- Очень близко расположенные к центру города отели показывают более высокий рейтинг
- Расстояние до аэропорта оказывает влияние на рейтинг отеля
- Очень близкие к аэропорту отели ценятся выше других отелей (например для бизнес поездок)
- В общем случае чем дальше от аэропорта тем оценка выше (снижение шума)
- Сетевые отели имеют рейтинг выше обычных

Характеристики рецензентов - `reviewer_nationality`, `total_number_of_reviews_reviewer_has_given`:
- Туристы из разных регионов мира ставят разные оценки отелю
- Туристы являющиеся резидентами и туристы из других стран ставят разные оценки отелю
- Опытный пользователь (большое количество отзывов) может ставить более строгие оценки по сравнению с новичком

Признаки отзывов - `negative_review`, `positive_review`,  `tags`, `review_total_positive_word_counts`, `review_total_negative_word_counts`:
- Наличие негативного отзыва оказывает существенное влияние на рейтинг
- Наличие позитивного отзыва оказывает существенное влияние на рейтинг
- Наличие определенных слов (паттернов) из отзывов и тегов оказывает влияние на рейтинг
- Разный тип комнат оценивается по разному
- Бизнес поездка и отдых (тип поездки) оцениваются по разному
- Количество дней отдыха оказывает влияние на оценку
- Короткий негативный отзыв как правило оценивается выше чем длинный
- Короткий позитивный отзыв как правило оценивается ниже чем длинный

Признаки отеля - `total_number_of_reviews` и `additional_number_of_scoring`, `average_score`:
- Большое количество отзывов отеля отличается от малого количества отзывов
- Большое количество молчаливых оценок оказывает влияение на рейтинг по сравнению с малым
- Возможно отели с малым количеством отзывов, и с малым количеством молчаливых оценок - новые, соответственно выше оценены.
- Доля молчаливых отзывов оказывает влияение на рейтинг (чем меньше доля тем честнее оценка, и возможно ниже)
- Категории популярности отеля оказывает влияение на рейтинг (чем популярнее тем выше оценка)

Все гипотезы будут проверены в следующем разделе после формирования новых признаков, способных ответить на поставленные вопросы.

### Даты - `review_date` и `days_since_review`

In [30]:
# Создаем новые признаки год, месяц, квартал отзыва
data['review_year'] = data['review_date'].dt.year.astype('category')
data['review_month'] = data['review_date'].dt.month.astype('category')
data['review_quarter'] = data['review_date'].dt.quarter.astype('category')

In [31]:
# Визуализируем результат
new_features = ['review_year', 'review_month', 'review_quarter']
titles = [
    'Распределение оценок по годам',
    'Распределение оценок по месяцам',
    'Распределение оценок по кварталам'
]
xlabels = ['Год', 'Месяц', 'Квартал']
get_boxplots(new_features, titles, xlabels)
# Для отображения раскомментировать
# plt.show()
plt.close()

# Выведем отдельно статистику по данным
get_stats('review_year')
get_stats('review_month')
get_stats('review_quarter')

Статистика reviewer_score по review_year:


,count,mean,std,min,25%,50%,75%,max
review_year,,,,,,,,
2015,70851.0,8.32,1.65,2.5,7.5,8.8,9.6,10.0
2016,198071.0,8.43,1.61,2.5,7.5,8.8,9.6,10.0
2017,117574.0,8.39,1.66,2.5,7.5,8.8,9.6,10.0


Статистика reviewer_score по review_month:


,count,mean,std,min,25%,50%,75%,max
review_month,,,,,,,,
1,31494.0,8.48,1.60,2.5,7.5,9.2,9.6,10.0
2,28717.0,8.54,1.56,2.5,7.5,9.2,9.6,10.0
3,30881.0,8.48,1.58,2.5,7.5,8.8,9.6,10.0
4,32202.0,8.43,1.61,2.5,7.5,8.8,9.6,10.0
5,34664.0,8.39,1.64,2.5,7.5,8.8,9.6,10.0
6,32207.0,8.39,1.65,2.5,7.5,8.8,9.6,10.0
7,37114.0,8.34,1.67,2.5,7.5,8.8,9.6,10.0
8,37871.0,8.41,1.63,2.5,7.5,8.8,9.6,10.0
9,31739.0,8.29,1.70,2.5,7.5,8.8,9.6,10.0


Статистика reviewer_score по review_quarter:


,count,mean,std,min,25%,50%,75%,max
review_quarter,,,,,,,,
1,91092.0,8.50,1.58,2.5,7.5,9.2,9.6,10.0
2,99073.0,8.40,1.63,2.5,7.5,8.8,9.6,10.0
3,106724.0,8.35,1.66,2.5,7.5,8.8,9.6,10.0
4,89607.0,8.34,1.65,2.5,7.5,8.8,9.6,10.0


<div align="center">
<image src="data/image/plt_1.png" height="400"></div>

Признак год отзыва демонстрирует небольшой тренд: от 8.32 (2015) до 8.43 (2016) с последующим спадом до 8.39 (2017), разница составляет 0.11 балла. Для месяца наблюдается выраженная сезонность с пиком в феврале (8.54) и спадом в октябре (8.26) - разница 0.28 балла. Кварталы показывают последовательное снижение оценок от 8.50 к 8.34. Нужно подтвердить, что различия статистически значимы и проверить на корреляцию между собой.

In [32]:
# Проверка признака days_since_review
reference_date = data['review_date'].max()
timedelta_days = pd.to_timedelta(data['days_since_review'], unit='D')
calculated_review_date = reference_date - timedelta_days

# Сравниваем с фактической review_date
dates_match = calculated_review_date.dt.date == data['review_date'].dt.date

print(f"Процент совпадений: {dates_match.mean():.2%}")

Процент совпадений: 100.00%


In [33]:
# Гипотеза - свежесть отзыва оказывает влияние на рейтинг отеля
data['since_review'] = pd.cut(
    data['days_since_review'],
    bins=[0, 90, 365, float('inf')],
    labels=['3_months', '1_year', 'old'],
    include_lowest=True
)

# Бинарный признак - совсем свежие отзывы (< 30 дней)
data['recent_review'] = (data['days_since_review'] < 30).astype(int)

# Посмотрим статистику
get_stats('since_review')
get_stats('recent_review')

Статистика reviewer_score по since_review:


,count,mean,std,min,25%,50%,75%,max
since_review,,,,,,,,
3_months,52097.0,8.33,1.70,2.5,7.5,8.8,9.6,10.0
1_year,148912.0,8.40,1.65,2.5,7.5,8.8,9.6,10.0
old,185487.0,8.41,1.60,2.5,7.5,8.8,9.6,10.0


Статистика reviewer_score по recent_review:


,count,mean,std,min,25%,50%,75%,max
recent_review,,,,,,,,
0,368681.0,8.40,1.63,2.5,7.5,8.8,9.6,10.0
1,17815.0,8.31,1.70,2.5,7.5,8.8,9.6,10.0


In [34]:
# Удаляю отработанный признак
data.drop('review_date', axis=1, inplace=True)

### Локация - `hotel_address`, `hotel_name`, `lat`, `lng`

Мы знаем, что у нас имеется разница в количестве уникальных значений между адресом и названием отеля. Прежде чем продолжать, исправим это:

In [35]:
# Группируем по названию 
name_address = data.groupby('hotel_name')['hotel_address'].nunique()
# Отели с одинаковыми названиями в разных местах
same_name_hotels = name_address[name_address > 1]
same_name_hotels

hotel_name
Hotel Regina    3
Name: hotel_address, dtype: int64

In [36]:
data[data['hotel_name'] == same_name_hotels.index[0]]['hotel_address'].unique()

array(['Bergara 2 4 Eixample 08002 Barcelona Spain',
       'Rooseveltplatz 15 09 Alsergrund 1090 Vienna Austria',
       'Via Cesare Correnti 13 Milan City Center 20123 Milan Italy'],
      dtype=object)

In [37]:
# Исправлю название отеля по адресу
data.loc[
    data['hotel_address'] == "Bergara 2 4 Eixample 08002 Barcelona Spain", 
    'hotel_name'
] = "Hotel Regina Barcelona"

data.loc[
    data['hotel_address'] == "Rooseveltplatz 15 09 Alsergrund 1090 Vienna Austria", 
    'hotel_name'
] = "Hotel Regina Vienna"

data.loc[
    data['hotel_address'] == "Via Cesare Correnti 13 Milan City Center 20123 Milan Italy", 
    'hotel_name'
] = "Hotel Regina Milan"

Так как оказалось 3 отеля с одинаковым названием, очевидно, что данные снова не равномерны, значит есть отели по одному адресу с разным названием. Проверим:

In [38]:
# Группируем по адресу
address_name = data.groupby('hotel_address')['hotel_name'].nunique()
# Отели с одинаковыми названиями в разных местах
same_address_hotels = address_name[address_name > 1]
same_address_hotels

hotel_address
8 Northumberland Avenue Westminster Borough London WC2N 5BY United Kingdom    2
Name: hotel_name, dtype: int64

In [39]:
# Смотрим названия
same_address = data['hotel_address'] == same_address_hotels.index[0]
data[same_address]['hotel_name'].unique()

array(['Club Quarters Hotel Trafalgar Square',
       'The Grand at Trafalgar Square'], dtype=object)

Таким образом в данных имеется 3 отеля с одинаковым названием, и 2 отеля с разными названиями но в одном здании. 
Согласно гуглу: ранее отели были единым целым, в настоящее время они работают как самостоятельные гостиницы: The Grand at Trafalgar Square и Club Quarters Hotel Trafalgar Square. Хотя они имеют одинаковое расположение, у них есть свои особенности. Соответственно исправлять ничего не будем, на 1 уникальное значение имени будет больше, чем адресов.

In [40]:
# Проверяю
data[['hotel_address', 'hotel_name']].describe()

,hotel_address,hotel_name
count,515431,515431
unique,1493,1494
top,163 Marsh Wall Docklands Tower Hamlets London ...,Britannia International Hotel Canary Wharf
freq,4789,4789


Данные исправлены, начнем проектирование признаков с адреса:

In [41]:
# Посмотрим на данные, что бы понять что можем извлечь
data['hotel_address'].head(10).tolist()

['Via Senigallia 6 20161 Milan Italy',
 'Arlandaweg 10 Westpoort 1043 EW Amsterdam Netherlands',
 'Mallorca 251 Eixample 08008 Barcelona Spain',
 'Piazza Della Repubblica 17 Central Station 20124 Milan Italy',
 'Singel 303 309 Amsterdam City Center 1012 WJ Amsterdam Netherlands',
 'Coram Street Camden London WC1N 1HT United Kingdom',
 'Empire Way Wembley Brent London HA9 8DS United Kingdom',
 '1 Shortlands Hammersmith and Fulham London W6 8DR United Kingdom',
 '35 Rue Caumartin 9th arr 75009 Paris France',
 '49 Gloucester Place Marble Arch Westminster Borough London W1U 8JE United Kingdom']

Извлечем страну и город из адреса. Судя по структуре представленных данных, наименование страны является последним словом в строке, кроме "United Kingdom" - 2 слова. Наименование города, предпоследнее слово, кроме United Kingdom, где London на 5 позиции с конца. Но я не уверена, что в данных только London, поэтому сделаю поиск по позиции.

In [42]:
# Применяем преобразования ко всему столбцу hotel_address
data['country'] = data['hotel_address'].apply(
    lambda x: 'United Kingdom' if 'Kingdom' in x.split()[-1] else x.split()[-1]
)
data['city'] = data['hotel_address'].apply(
    lambda x: x.split()[-5] if 'Kingdom' in x.split()[-1] else x.split()[-2]
)

hotel_counts = data.groupby(['country', 'city']).size().reset_index(name='count')
display(hotel_counts)

,country,city,count
0,Austria,Vienna,38938
1,France,Paris,59626
2,Italy,Milan,37206
3,Netherlands,Amsterdam,57212
4,Spain,Barcelona,60149
5,United Kingdom,London,262300


In [43]:
# Визуализируем полученные данные
fig = get_boxplots(['country'], ['Рейтинг в зависимости от страны'], ['Страна'])
fig.axes[0].tick_params(axis='x', rotation=45)
# Для отображения раскомментировать
# plt.show()
plt.close()

# Выведем отдельно статистику по данным
get_stats('country')

Статистика reviewer_score по country:


,count,mean,std,min,25%,50%,75%,max
country,,,,,,,,
Austria,29177.0,8.55,1.51,2.5,7.9,9.2,9.6,10.0
France,44528.0,8.42,1.65,2.5,7.5,8.8,9.6,10.0
Italy,27882.0,8.36,1.64,2.5,7.5,8.8,9.6,10.0
Netherlands,43004.0,8.45,1.60,2.5,7.5,8.8,9.6,10.0
Spain,45132.0,8.55,1.56,2.5,7.9,9.2,9.6,10.0
United Kingdom,196773.0,8.33,1.67,2.5,7.5,8.8,9.6,10.0


<div align="center">
<image src="data/image/plt_2.png" height="400"></div>

В данных всего 6 городов из 6 стран, соответственно оставлять оба признака бессмысленно, достаточно одного признака страны. Средние рейтинги по странам варьируются от 8.33 (United Kingdom) до 8.55 (Austria, Spain) - разница составляет 0.22 балла. United Kingdom резко выделяется по количеству отзывов при этом демонстрирует наиболее низкий средний рейтинг. Необходимо провести статистический тест на различие между странами, возможно объединить некоторые, которые не дают значимых различий, в целях уменьшения количества признаков.

Прежде чем удалить признак города, извлечем из него полезную информацию для нас с помощью координат. Расстояние от центра города, расстояние от аэропорта.

In [44]:
# Координаты центров городов
city_centers = {
    'Vienna': (48.2082, 16.3738),
    'Paris': (48.8566, 2.3522),
    'Milan': (45.4642, 9.1900),
    'Amsterdam': (52.3676, 4.9041),
    'Barcelona': (41.3851, 2.1734),
    'London': (51.5074, -0.1278)
}

# Координаты аэропортов
airports = {
    'Vienna': (48.1103, 16.5697),    # VIE
    'Paris': (49.0097, 2.5479),      # CDG
    'Milan': (45.6300, 8.7281),      # MXP
    'Amsterdam': (52.3081, 4.7642),  # AMS
    'Barcelona': (41.2974, 2.0833),  # BCN
    'London': (51.4700, -0.4543)     # LHR
}

# Считаем расстояния только для уникальных координат отелей,
# это ускорит работу с библиотекой
unique_hotels = data[['city', 'lat', 'lng']].drop_duplicates()

# Вычисляем расстояния для каждого отеля
hotel_distances = {}
for _, hotel in unique_hotels.iterrows():
    city = hotel['city']
    coord = (hotel['lat'], hotel['lng'])
    
    dist_center = geodesic(coord, city_centers[city]).km
    dist_airport = geodesic(coord, airports[city]).km
    
    hotel_distances[(hotel['lat'], hotel['lng'])] = (dist_center, dist_airport)

# Создаем DataFrame с расстояниями
distances_df = pd.DataFrame([
    {'lat': lat, 'lng': lng, 'dist_center': dist_c, 'dist_airport': dist_a}
    for (lat, lng), (dist_c, dist_a) in hotel_distances.items()
])

# Объединяем таблицы
data = data.merge(distances_df, on=['lat', 'lng'], how='left')

display(data[['dist_center', 'dist_airport']].describe())

,dist_center,dist_airport
count,515431.000000,515431.000000
mean,3.155241,21.007486
std,2.555966,7.693838
min,0.075445,4.243286
25%,1.361589,16.325844
50%,2.471499,20.865825
75%,4.088283,23.756684
max,17.189429,45.989687


In [45]:
# Определим данный признак в категории с помощью квантилей
# Для центра
data['dist_center_cat'] = pd.qcut(
    data['dist_center'], q=4,
    labels=['Очень близко', 'Близко', 'Далеко', 'Очень далеко']
)

# Для аэропорта
data['dist_airport_cat'] = pd.qcut(
    data['dist_airport'], q=4,
    labels=['Очень близко', 'Близко', 'Далеко', 'Очень далеко']
)

# Визуализируем полученные данные
new_features = ['dist_center_cat', 'dist_airport_cat']
titles = ['Рейтинг по удаленности от центра', 'Рейтинг по удаленности от аэропорта']
xlabels = ['Категория удаленности от центра', 'Категория удаленности от аэропорта']
get_boxplots(new_features, titles, xlabels)

# Для отображения раскомментировать
# plt.show()
plt.close()

# Статистика по данным
get_stats('dist_center_cat')
get_stats('dist_airport_cat')

Статистика reviewer_score по dist_center_cat:


,count,mean,std,min,25%,50%,75%,max
dist_center_cat,,,,,,,,
Очень близко,96810.0,8.65,1.51,2.5,7.9,9.2,10.0,10.0
Близко,96813.0,8.42,1.61,2.5,7.5,8.8,9.6,10.0
Далеко,96391.0,8.31,1.67,2.5,7.5,8.8,9.6,10.0
Очень далеко,96482.0,8.21,1.72,2.5,7.1,8.8,9.6,10.0


Статистика reviewer_score по dist_airport_cat:


,count,mean,std,min,25%,50%,75%,max
dist_airport_cat,,,,,,,,
Очень близко,97039.0,8.51,1.57,2.5,7.5,9.2,9.6,10.0
Близко,96517.0,8.16,1.71,2.5,7.1,8.8,9.6,10.0
Далеко,97237.0,8.42,1.62,2.5,7.5,8.8,9.6,10.0
Очень далеко,95703.0,8.49,1.62,2.5,7.5,9.2,9.6,10.0


<div align="center">
<image src="data/image/plt_3.png" height="400"></div>

Различия в средних рейтингах между категориями удаленности от центра прослеживаются четко. Для удаленности от аэропорта моя гипотеза подтверждается,  проседание для категории - "близко", при этом "очень близко" и "очень далеко" показали максимальные значения. Причина явно в целях туристов, для краткосрочных бизнес поездок или пересадок, близость к аэропорту может стать ключевым фактором выбора отеля, тогда как для долгосрочных путешествий или путешествий с детьми, наоборот выбирают отели подальше от аэропорта, во избежание шума от него. При дальнейшем анализе необходимо проверить есть ли статистически значимое влияние на рейтинг, и принять решение оставлять признак расстояния в числовом формате, или использовать категориальный формат.

Теперь рассмотрим признак название отеля, и что можно из него извлечь:

In [46]:
# Посмотрим на данные, что бы понять что можем извлечь
data['hotel_name'].head(10).tolist()

['Hotel Da Vinci',
 'Urban Lodge Hotel',
 'Alexandra Barcelona A DoubleTree by Hilton',
 'Hotel Principe Di Savoia',
 'Hotel Esther a',
 'Holiday Inn London Bloomsbury',
 'Holiday Inn London Wembley',
 'Novotel London West',
 'Hotel Saint Petersbourg Opera',
 'St George Hotel']

In [47]:
# Определяю ключевые слова для поиска сетевых отелей
hotel_chain = [
    'Hilton', 'Marriott', 'Sheraton', 'Holiday Inn', 'Radisson',
    'Hyatt', 'Novotel', 'Mercure', 'Ibis', 'Accor', 'Renaissance',
    'Westin', 'Aloft', 'Adagio', 'Four Seasons', 'Best Western',
    'Movenpick', 'Barcelo', 'Leonardo', 'Element', 'Millennium',
    'Ramada', 'InterContinental'
]

data['chain_hotel'] = data['hotel_name'].apply(
    lambda x: 1 if any(chain in x for chain in hotel_chain) else 0
).astype('int8')
data['chain_hotel'].value_counts()

chain_hotel
0    377840
1    137591
Name: count, dtype: int64

In [48]:
# Визуализируем полученные данные
new_features = ['chain_hotel']
titles = ['Распределение рейтингов: сетевые vs независимые отели']
fig = get_boxplots(new_features, titles)
fig.axes[0].set_xticks([0, 1])
fig.axes[0].set_xticklabels(['Независимые', 'Сетевые'])

# Для отображения раскомментировать
# plt.show()
plt.close()
# Статистика по данным

# Выведем отдельно статистику по данным
get_stats('chain_hotel')

Статистика reviewer_score по chain_hotel:


,count,mean,std,min,25%,50%,75%,max
chain_hotel,,,,,,,,
0,283164.0,8.46,1.61,2.5,7.5,8.8,9.6,10.0
1,103332.0,8.21,1.68,2.5,7.1,8.8,9.6,10.0


<div align="center">
<image src="data/image/plt_4.png" height="400"></div>

Сетевые отели демонстрируют более низкий средний рейтинг (8.21) по сравнению с независимыми отелями (8.46). Разница составляет 0.25 балла, неожиданно и опровергает первоначальную гипотезу о высоком качестве сетевых отелей. Возможно связано с более высокими ожиданиями у отдыхающих. Необходимо провести статистические тест оказывает ли влияние на целевую переменную данный признак.

Итого, созданы признаки страна, город, расстояние до центра и аэропорта в км. и по категориям, бинарный признак сетевой ли отель. На данном этапе, у меня больше нет идей какие гипотезы можно выдвинуть, возможно они появятся после проведения статистических тестов. Пока перехожу к следующему разделу. Признакам расстояния поменяю тип данных. Страну переведу в категориальный тип. Отработанные признаки удаляю (город, имя отеля, адрес, координаты).

In [49]:
# Смена типа данных
cols_to_float32 = ['dist_center', 'dist_airport']
cols_to_category = ['country', 'city']
for col in cols_to_float32:
    data[col] = data[col].astype('float32')
    
for col in cols_to_category:
    data[col] = data[col].astype('category')

# Удаляю отработанные признаки, и не нужный города
data.drop(['hotel_address', 'lng', 'lat', 'hotel_name', 'city'], axis=1, inplace=True)

### Характеристики рецензентов - `reviewer_nationality`, `total_number_of_reviews_reviewer_has_given`

In [50]:
# Посмотрим на данные, что бы понять содержание
data['reviewer_nationality'].head(10).tolist()

['United Kingdom',
 'Belgium',
 'Sweden',
 'United States of America',
 'United Kingdom',
 'Ecuador',
 'United Kingdom',
 'Netherlands',
 'Ireland',
 'Canada']

Как уже ранее было отмечено, в данном признаке слишком много уникальных значений, соответственно моя гипотеза сформулирована так: туристы из разных регионов мира ставят разные оценки. Для ответа на поставленный вопрос необходимо переопределить данные в регионы. Необходимо также отметить, что в данных имеются некоторые странны, которые не находятся ни одним словарем ('Macau', 'Kosovo', 'Palestinian Territory', 'Cura ao'), количество отзывов от них довольно маленькое, поэтому мной принято решение сначала "почистить" данные, я оставляю 95% уникальных значений, остальные страны определю как "Other".

In [51]:
# Топ национальности покрывающие 95% данных
top_nationalities = data['reviewer_nationality'].value_counts().cumsum() / len(data)
top_95 = top_nationalities[top_nationalities <= 0.95].index

# Создаем признак с lambda
data['reviewer_nationality'] = data['reviewer_nationality'].apply(
    lambda x: x if x in top_95 else 'Other'
).astype('category')
print(f"В данных осталось {data['reviewer_nationality'].nunique()} национальность")

В данных осталось 51 национальность


Данные были уменьшены в 5 раз. Теперь попробуем выделить принадлежность к региону (Европа, Америка, Азия и т.п.) через справочник "ISO-3166-Countries-with-Regional-Codes" найденном на просторах GitHub. После проверю, все ли страны нашлись в справочнике, если чего-то не хватает, исправлю вручную.

In [52]:
# Создадим список национальностей
nationality = data['reviewer_nationality'].value_counts().index.to_list()

# DataFrame с данными ИСО 3166
url = 'https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/refs/heads/master/all/all.csv'
country_df = pd.read_csv(url, usecols=['name', 'region'])

# Создаем словарь с нужными нам данными
map_region = {}
for nat in nationality:
    mask = country_df['name'].str.contains(nat)
    if mask.any():
        map_region[nat] = country_df[mask]['region'].iloc[0]
    else:
        map_region[nat] = 0

# Проверяем все ли страны нашлись
region_with_zero = [country for country, region in map_region.items() if region == 0]
print(f'Cтраны которые не нашлись в словаре {region_with_zero}')

Cтраны которые не нашлись в словаре ['Other', 'Turkey', 'Czech Republic']


In [53]:
# Добавим 'Turkey', 'Czech Republic' и 'Other' вручную
map_region = {
    **map_region, **{'Turkey': 'Asia', 'Czech Republic': 'Europe', 'Other': 'Other'}
}

# Создаем новые признаки из словарей
data['region_nationality'] = (
    data['reviewer_nationality'].map(map_region).astype('category')
)

# Визуализируем полученный признак
get_boxplots(
    ['region_nationality'],
    ['Средние рейтинги по региону национальности'],
    ['Регион национальности гостя'])

# Для отображения раскомментировать plt.show()
# plt.show()
plt.close()

# Статистика по данным
get_stats('region_nationality')

Статистика reviewer_score по region_nationality:


,count,mean,std,min,25%,50%,75%,max
region_nationality,,,,,,,,
Africa,7224.0,8.13,1.75,2.5,7.1,8.3,9.6,10.0
Americas,33860.0,8.73,1.51,2.5,7.9,9.2,10.0,10.0
Asia,46033.0,8.10,1.76,2.5,7.1,8.3,9.6,10.0
Europe,261078.0,8.41,1.62,2.5,7.5,8.8,9.6,10.0
Oceania,18659.0,8.60,1.47,2.5,7.9,9.2,9.6,10.0
Other,19642.0,8.23,1.67,2.5,7.5,8.8,9.6,10.0


<div align="center">
<img src="data/image/plt_5.png" height="400"></div>

По графику видно, что рейтинг отелей в зависимости от региона проживания гостя различен, самые большой рейтинг у отелей, где преимущественно останавливаются Американцы, самые низкие рейтинги у отелей где останавливаются Африканцы и Азиаты. Необходимо провести статистический тест на различие рейтингов в группах, возможно некоторые группы можно объединить в одну.

In [54]:
# Туристы являющиеся резидентами и туристы из других стран ставят разные оценки отелю
# Создаем бинарный признак является ли гость отеля туристом
# 1 - гость, 0 - резидент
data['is_tourist'] = (
    data['country'].astype(str) != data['reviewer_nationality'].astype(str)
).astype('int8')

# Посмотрим статистику
get_stats('is_tourist')

Статистика reviewer_score по is_tourist:


,count,mean,std,min,25%,50%,75%,max
is_tourist,,,,,,,,
0,133922.0,8.42,1.65,2.5,7.5,8.8,9.6,10.0
1,252574.0,8.39,1.63,2.5,7.5,8.8,9.6,10.0


In [55]:
# Опытный пользователь может ставить более строгие оценки по сравнению с новичком
# Создаем признак на основе квантилей
data['reviewer_experience'] = pd.qcut(
    data['total_number_of_reviews_reviewer_has_given'],
    q=3, labels=['Новый пользователь', 'Активный пользователь', 'Эксперт']
)
get_stats('reviewer_experience')

# Создадим бинарный признак опытного пользователя, который написал больше 20 отзывов
data['expert_reviewer'] = data['total_number_of_reviews_reviewer_has_given'].apply(
    lambda x: 1 if x > 20 else 0).astype('int8')
# Посмотрим статистику
get_stats('expert_reviewer')

Статистика reviewer_score по reviewer_experience:


,count,mean,std,min,25%,50%,75%,max
reviewer_experience,,,,,,,,
Новый пользователь,166251.0,8.40,1.70,2.5,7.5,8.8,9.6,10.0
Активный пользователь,98813.0,8.39,1.64,2.5,7.5,8.8,9.6,10.0
Эксперт,121432.0,8.40,1.53,2.5,7.5,8.8,9.6,10.0


Статистика reviewer_score по expert_reviewer:


,count,mean,std,min,25%,50%,75%,max
expert_reviewer,,,,,,,,
0,355946.0,8.40,1.65,2.5,7.5,8.8,9.6,10.0
1,30550.0,8.41,1.47,2.5,7.5,8.8,9.6,10.0


Гипотезу о туристах и резидентах тяжело подтвердить по полученным данным, разница в рейтинге не прослеживается. Также нет разницы в рейтинге для категорий пользователей по количеству оставленных отзывов, попробовала выделить пользователей оставивших больше 20 отзывов в бинарный признак, но также не вижу разницы, необходимо провести тесты и избавится от признаков, так как гипотезы скорее всего не подтвердятся. Также необходимо проверить корреляцию с первоначального признака 'total_number_of_reviews_reviewer_has_given' с целевой переменной, скорее всего корреляция слабая, и признак бесполезен для модели.

In [56]:
# Удаляю отработанный признак
data.drop('reviewer_nationality', axis=1, inplace=True)

### Признаки отзывов

#### Анализ признака `negative_review`

In [57]:
# Посмотрим на данные, что бы понять содержание
data['negative_review'].value_counts().head(10)

negative_review
no negative       127816
nothing            21177
n a                 1815
none                1501
                     849
nothing really       655
small room           570
breakfast            545
all good             544
no complaints        523
Name: count, dtype: int64

Как видно по самым частым включениям в отзывах, помимо фразы "No Negative", имеются фразы типа Nothing, None, N/A, которые не несут негативный контекст, а лишь означают, что нет "плохих" сторон у отеля, необходимо переопределить их в "No Negative"

In [58]:
# Список не негативных фраз довольно обширный
# Вычислю все, что больше 100
data['negative_review'].value_counts().loc[lambda x: x > 100]

negative_review
no negative                       127816
nothing                            21177
n a                                 1815
none                                1501
                                     849
nothing really                       655
small room                           570
breakfast                            545
all good                             544
no complaints                        523
location                             501
nothing at all                       466
everything                           419
nothing to dislike                   373
price                                348
nil                                  332
small rooms                          307
everything was perfect               276
na                                   272
can t think of anything              250
leaving                              244
absolutely nothing                   242
everything was great                 222
expensive                            206


In [59]:
# Составила список самых частых фраз, означающих "нет негатива"
non_negative_phrases = [
    'no negative', 'nothing', 'none', 'n a', 'na', 'nothing really', '',
    'nothing at all', 'nothing to dislike', 'nil', 'all good', 'no complaints',
    'everything was perfect', 'can t think of anything', 'absolutely nothing',
    'everything was great', 'nothing to complain about', 'no', 'nothing not to like',
    'nothing all good', 'everything was good', 'i liked everything', 'liked everything',
    'nothing it was perfect', 'non', 'nothing everything was perfect',
    'everything was fine', 'everything was fine', 'nothing everything was great',
    'all was good', 'nothing comes to mind', 'loved everything', 'it was all good'
    'nothing i didn t like', 'no negatives', 'there was nothing i didn t like',
    'no complaints at all', 'we liked everything', 'all ok', 'nothing to report',
    'no thing', 'there was nothing to dislike', 'nothing to say', 'no dislikes',
    'no comment', 'no bad experience', 'nothing to complain', 'no issues',
    'there was nothing not to like', 'everything was ok', 'everything was excellent',
    'nothing we didn t like', 'no problems', 'nothing bad to say'
]

# Объединяем все в 'no negative' в исходном признаке
data['negative_review'] = data['negative_review'].apply(
    lambda x: 'no negative' if x in non_negative_phrases else x
)

# Создаем бинарный признак наличия негативного отзыва
data['has_negative'] = (data['negative_review'] 
                        != 'no negative').astype('int8')

Для ответа на сформулированную гипотезу: наличие определенных слов (паттернов) из отзывов оказывает влияние на рейтинг. Необходимо выделить бинарные признаки, по различным ключевым словам в отзыве:

In [60]:
# 1. Недовольство комнатой
room_keywords = [
    'room', 'rooms', 'small', 'size', 'tiny', 'space',
    'bed', 'beds', 'pillow', 'mattress', 'uncomfortable', 'hard',
    'bathroom', 'bath', 'shower', 'toilet', 'water', 'cold', 'hot',
    'window', 'windows', 'view', 'noise', 'noisy', 'quiet',
    'furniture', 'wardrobe', 'closet', 'safe', 'drawers',
    'minibar', 'mini bar', 'bar', 'fridge', 'refrigerator',
    'tv', 'television', 'air conditioning', 'ac', 'heating',
    'wifi', 'wi-fi', 'internet', 'connection', 'working'
]
data['bad_room'] = data['negative_review'].str.contains(
    '|'.join(room_keywords), case=False).astype('int8')

# 2. Недовольство обслуживанием и персоналом
service_keywords = [
    'staff', 'service', 'reception', 'desk', 'rude',
    'clean', 'cleaning', 'dirty', 'dust', 'stain', 'housekeeping',
    'slow', 'wait', 'time', 'check in', 'check out'
]
data['bad_service'] = data['negative_review'].str.contains(
    '|'.join(service_keywords), case=False).astype('int8')

# 3. Недовольство едой и напитками
food_keywords = [
    'breakfast', 'food', 'dinner', 'lunch', 'meal',
    'restaurant', 'coffee', 'tea', 'drink',
    'buffet', 'menu'
]
data['bad_food'] = data['negative_review'].str.contains(
    '|'.join(food_keywords), case=False).astype('int8')

# 4. Недовольство ценой
price_keywords = [
    'price', 'expensive', 'cost', 'overpriced', 
    'money', 'paid', 'pay', 'extra', 'value'
]
data['bad_price'] = data['negative_review'].str.contains(
    '|'.join(price_keywords), case=False).astype('int8')

# 5. Недовольство местоположением
location_keywords = [
    'location', 'area', 'far', 'distance', 'walk',
    'noisy', 'street', 'city'
]
data['bad_location'] = data['negative_review'].str.contains(
    '|'.join(location_keywords), case=False, na=False
).astype('int8')

# 6. Недовольство дополнительными услугами отеля
hotel_option_keywords = [
    'pool', 'swimming', 'gym', 'fitness', 'parking',
    'facilities', 'lift', 'elevator'
]
data['bad_option'] = data['negative_review'].str.contains(
    '|'.join(hotel_option_keywords), case=False).astype('int8')

In [61]:
# Список новых бинарных признаков
new_features = ['has_negative', 'bad_room', 'bad_service', 'bad_food', 
                'bad_price', 'bad_location', 'bad_option']
titles = [
    'Наличие жалоб', 'Проблемы с номером', 'Проблемы с обслуживанием', 
    'Проблемы с едой', 'Недовольство ценой', 'Проблемы с местоположением',
    'Проблемы с услугами отеля'
]
fig = get_boxplots(new_features, titles, ncols=2)
for i, ax in enumerate(fig.axes[:len(new_features)]):
    ax.set_xticks([0, 1])
    ax.set_xticklabels(['Нет', 'Есть'])

plt.tight_layout()
# Для отображения раскомментировать
# plt.show()
plt.close()

<div align="center">
<image src="data/image/plt_7.png" width="800"></div>

Анализ негативных отзывов выявил, что наличие жалоб значительно снижает средний рейтинг. Кроме того по самым частым отзывам были выделены некоторые категории недовольства, все из них показали на графике значительную разницу в средних баллах. На этапе отбора признаков, необходимо будет проверить статистическую значимость влияния на рейтинг, корреляцию данных признаков между собой, но думаю в целом это сильные признаки, для обучения модели. 

Первоначальный признак отзыва нам больше не понадобится, поэтому сразу удаляю его:

In [62]:
data.drop(['negative_review'], axis=1, inplace=True)

#### Анализ признака `positive_review`

In [63]:
# Посмотрим на данные, что бы понять содержание
data['positive_review'].value_counts().head(10)

positive_review
no positive           35924
location              11933
everything             3314
nothing                1729
great location         1685
the location           1604
good location          1495
breakfast               795
friendly staff          732
excellent location      639
Name: count, dtype: int64

Пойдем по тому же пути что и с негативными, найдем все "не позитивные" отзывы.

In [64]:
# Фразы, которые означают отсутствие позитивного отзыва
non_positive_phrases = [
    'no positive', 'nothing', 'none', 'nothing really', 'nothing at all', 'na', ''
]

# Объединяем все в 'no positive' в исходном признаке
data['positive_review'] = data['positive_review'].apply(
    lambda x: 'no positive' if x in non_positive_phrases else x
)

# Создаем бинарный признак наличия негативного отзыва
data['has_positive'] = (data['positive_review'] != 'no positive').astype('int8')

# Проверяем результат
data['positive_review'].value_counts().loc[lambda x: x > 200]

positive_review
no positive                   38001
location                      11933
everything                     3314
great location                 1685
the location                   1604
good location                  1495
breakfast                       795
friendly staff                  732
excellent location              639
staff                           553
location and staff              452
location staff                  432
location location location      345
location was great              328
good breakfast                  316
everything was perfect          316
location was good               312
comfy bed                       303
perfect location                287
the staff                       280
location is great               278
location is good                256
all                             250
clean                           237
every thing                     235
very friendly staff             233
great breakfast                 221
breakfast wa

Отзывов где люди указали положительные комментарии куда больше, чем негативных. Возможно признак (наличие позитивного отзыва) не покажет статистическую значимость. Поступим далее по тому же пути, что и с негативными отзывами, получим бинарные признаки из частых фраз/слов.

In [65]:
# 1. Удовлетворенность локацией
location_keywords = [
    'location', 'area', 'walk','center','centre','downtown','metro', 'station',
    'transport','subway','underground'
]
data['good_location'] = data['positive_review'].str.contains(
    '|'.join(location_keywords), case=False).astype('int8')

# 2. Удовлетворенность персоналом
staff_keywords = [
    'staff', 'service', 'reception', 'housekeeping', 'check in', 'check out'
]
data['good_staff'] = data['positive_review'].str.contains(
    '|'.join(staff_keywords), case=False).astype('int8')

# 3. Удовлетворенность чистотой
clean_keywords = ['clean', 'cleaning']
data['good_clean'] = data['positive_review'].str.contains(
    '|'.join(clean_keywords), case=False).astype('int8')

# 4. Удовлетворенность завтраком/едой
food_keywords = [
    'breakfast', 'food', 'dinner', 'lunch', 'meal', 'restaurant',
    'coffee', 'tea', 'drink', 'buffet', 'menu'
]
data['good_food'] = data['positive_review'].str.contains(
    '|'.join(food_keywords), case=False).astype('int8')

# 5. Удовлетворенность комнатой и кроватью
positive_room_keywords = [
    'room', 'rooms', 'bed', 'pillow', 'mattress', 'bathroom', 'bath', 'shower',
    'toilet', 'water', 'window', 'windows', 'view', 'cozy', 'quiet', 'furniture',
    'wardrobe', 'closet', 'safe', 'drawers', 'minibar', 'mini bar', 'bar', 
    'fridge', 'refrigerator', 'tv', 'television', 'air conditioning', 'ac', 'heating',
    'wifi', 'wi-fi', 'internet', 'connection'
]
data['good_room'] = data['positive_review'].str.contains(
    '|'.join(room_keywords), case=False).astype('int8')

# 6. Общее удовлетворение (все отлично)
overall_keywords = [
    'everything', 'perfect', 'great', 'excellent', 'all', 'every thing', 'every',
    'awesome', 'amazing', 'fantastic', 'wonderful', 'brilliant'
]
data['good_overall'] = data['positive_review'].str.contains(
    '|'.join(overall_keywords), case=False).astype('int8')


In [66]:
# Список новых признаков
new_features = [
    'has_positive', 'good_location', 'good_staff', 'good_food',
    'good_room', 'good_overall', 'good_clean']
titles = [
    'Наличие позитивного отзыва', 'Наличие позитивного отзыва о местоположении', 
    'Наличие позитивного отзыва об обслуживании',
    'Наличие позитивного отзыва о еде', 'Наличие позитивного отзыва о номере',
    'Наличие в отзыве общих слов восхищения', 'Наличие позитивного отзыва о чистоте'
]
fig = get_boxplots(new_features, titles, ncols=2, colors=True)
for i, ax in enumerate(fig.axes[:len(new_features)]):
    ax.set_xticks([0, 1])
    ax.set_xticklabels(['Нет', 'Есть'])

plt.tight_layout()
# Для отображения раскомментировать
# plt.show()
plt.close()

<div align="center">
<image src="data/image/plt_8.png" width="800"></div>

Анализ позитивных отзывов показал, что 92.5% отзывов содержат положительные комментарии. Наличие позитивного отзыва значительно повышает средний рейтинг отеля. Кроме того было обнаружено влияние некоторых фраз на общий рейтинг, упоминание комнаты, персонала и общего тона отзыва с восхищением значительно повышает рейтинг. Признаки удовлетворенности от чистоты и еды также повышают рейтинг, но не так значимо как другие характеристики. Упоминание местоположения - самый частый отзыв, практически не оказывает положительного эффекта (в отличие от его отрицательного собрата), думаю люди часто выбирают удобные по местоположению отели, в ущерб другим благам, поэтому местоположение и не оказывает сильного эффекта на рейтинг. В разделе отбора признаков необходимо будет проверить как коррелируют признаки между собой и провести тесты влияния на рейтинг, для решения какие признаки оставлять для модели. Можно удалять исходный признак, и переходить к следующему.

In [67]:
data.drop(['positive_review'], axis=1, inplace=True)

#### Анализ признака `tags`

In [68]:
# Посмотрим на данные, что бы понять содержание
data['tags'].head(5)

0    Leisure trip, Couple, Double Room, Stayed 2 ni...
1     Leisure trip, Group, Triple Room, Stayed 1 night
2    Business trip, Solo traveler, Twin Room, Staye...
3    Leisure trip, Couple, Ambassador Junior Suite,...
4    Business trip, Solo traveler, Classic Double o...
Name: tags, dtype: object

In [69]:
# Посмотрим какие теги можно извлечь из списков
data['tags'] = data['tags'].str.split(', ')
data['tags'].explode().value_counts().head(15)

tags
Leisure trip                      417538
Submitted from a mobile device    307470
Couple                            252128
Stayed 1 night                    193568
Stayed 2 nights                   133850
Solo traveler                     108476
Stayed 3 nights                    95761
Business trip                      82884
Group                              65361
Family with young children         60989
Stayed 4 nights                    47777
Double Room                        35207
Standard Double Room               32247
Superior Double Room               31361
Family with older children         26334
Name: count, dtype: int64

На какие признаки можно разобрать теги, и посмотреть влияет ли на рейтинг:
- Бинарный признак is_mobile для тега "Submitted from a mobile device";
- Категориальный признак 'trip_type': "Leisure trip" - отдых, "Business trip" - бизнес поездка, "Other";
- Категориальный признак 'travelers_type': "Couple", "Solo traveler", "Group", "Family with young children", "Family with older children", "Travelers with friends";
- В данных есть теги означающий тип кровати: "Queen", "King", "Double", "Twin", но смущает что, например "Double", также имеет отношение к типу комнат. Поэтому создам признак наличия кровати 'king_size_bed', по тегам  "Queen" и "King";
- Интересные теги найдены "without Window". Не думаю что таких данных много, но создам для интереса бинарный признак отсутствия окна;
- Количество дней отдыха. Попробую разделить на категории, короткая бизнес поездка (1-3), стандартная (3-14) и длительная (более 14);
- Категориальный признак 'room_type'. Очень много типов комнат в данных, сначала выделю весь список, потом определю в три-четыре категории.

In [70]:
# Определю функцию для поиска тегов
def find_tags(tags_list: list, patterns: list) -> str:
    """Ищет частичное вхождение любого паттерна в тегах.
    
    Args:
        tags_list (list): Список тегов для поиска
        patterns (list): Список паттернов для поиска в тегах
        
    Returns:
        str: Найденный паттерн или 'other', если совпадений нет
    """
    # Заранее приводим все паттерны к нижнему регистру
    patterns_lower = [pattern.lower() for pattern in patterns]
    
    for tag in tags_list:
        tag_lower = str(tag).lower()
        
        # Ищем частичное совпадение каждого паттерна в теге
        for i, pattern_lower in enumerate(patterns_lower):
            if pattern_lower in tag_lower:
                return patterns[i]  # Возвращаем оригинальный паттерн
    
    return 'other'

In [71]:
# Бинарные признаки
# Бинарный признак is_mobile
data['is_mobile'] = data['tags'].apply(
    lambda x: 1 if 'Submitted from a mobile device' in x else 0
).astype('int8')

# Бинарный признак king_size_bed
data['king_size_bed'] = data['tags'].apply(
    lambda x: 1 if re.search(r'\b(King|Queen)\b', str(x)) else 0
).astype('int8')

# Бинарный признак without_window
window_keywords = ['without window', 'no window']

data['without_window'] = data['tags'].apply(
    lambda x: 0 if find_tags(x, window_keywords) == 'other' else 1
).astype('int8')

In [72]:
# Категориальные признаки
# Тип отдыха
trip_patterns = ['Leisure trip', 'Business trip']
data['trip_type'] = (
    data['tags'].apply(lambda x: find_tags(x, trip_patterns))
).astype('category')

# Тип отдыхающих
traveler_patterns = [
    'Couple', 'Solo traveler', 'Group', 'Family with young children', 
    'Family with older children', 'Travelers with friends'
]
data['travelers_type'] = (
    data['tags'].apply(lambda x: find_tags(x, traveler_patterns))
).astype('category')

# Тип комнаты
room_patterns = [
    'Standard', 'Superior', 'Deluxe', 'Classic', 'Economy', 'Basic', 'Comfort',
    'Executive', 'Club', 'Premium', 'Suite', 'Family Room', 'Apartment',
    'Guestroom', 'Luxury', 'Studio', 'Single Room', 'Double Room', 'Twin Room',
    'Triple Room'
]

data['room_type'] = data['tags'].apply(lambda x: find_tags(x, room_patterns))

# Зададим категории исходя из общепринятой практики, other в стандарт
room_categories = {
    "Эконом": ['Economy', 'Basic', 'Family Room', 'Apartment', 'Guestroom'],
    "Стандарт": [
        'Standard', 'Superior', 'Deluxe', 'Classic', 'Comfort', 'Executive',
        'Club', 'Studio', 'Single Room', 'Double Room', 'Twin Room', 
        'Triple Room', 'other'
    ],
    "Люкс": ['Suite', 'Luxury', 'Premium']
}

room_mapping = {
    room_type: category
    for category, rooms in room_categories.items() 
    for room_type in rooms
}
data['room_type'] = data['room_type'].map(room_mapping).astype('category')

In [73]:
# Количество дней отдыха
def extract_nights(tags_list):
    """ Извлекает количество ночей проживания из списка тегов.
    Тег представлен в формате 'Stayed X night[s]'.
    Arg:
        tags_list (list):  список тегов
    Returns
    int: количество ночей проживания"""
    for tag in tags_list:
        # Ищем тег, содержащий информацию о продолжительности проживания
        if 'Stayed' in tag and ('night' in tag or 'nights' in tag):
            # Разбиваем тег на слова и берем второе слово
            nights = tag.split()[1]
            return int(nights)
    # Возвращаем 0 если подходящий тег не найден
    return 0

data['nights_stayed'] = data['tags'].apply(extract_nights)
data['nights_stayed'] = data['nights_stayed'].astype('int8')
get_stats('nights_stayed')

Статистика reviewer_score по nights_stayed:


,count,mean,std,min,25%,50%,75%,max
nights_stayed,,,,,,,,
0,146.0,6.48,2.76,2.5,3.80,6.70,9.2,10.0
1,145296.0,8.41,1.66,2.5,7.50,8.80,9.6,10.0
2,100176.0,8.39,1.63,2.5,7.50,8.80,9.6,10.0
3,71940.0,8.43,1.58,2.5,7.50,8.80,9.6,10.0
4,35708.0,8.41,1.60,2.5,7.50,8.80,9.6,10.0
5,15592.0,8.34,1.63,2.5,7.50,8.80,9.6,10.0
6,7388.0,8.32,1.64,2.5,7.50,8.80,9.6,10.0
7,5541.0,8.30,1.68,2.5,7.50,8.80,9.6,10.0
8,1907.0,8.23,1.72,2.5,7.50,8.80,9.6,10.0


0 и более 14 ночей: слишком мало значений, с скачками рейтинга без явной логики (распределение точно не линейно).

Практически все данные распределены вокруг 1-7 ночей. Треть всех данных поездка на 1 ночь.

Была попытка поделить признак на различные категории длительности, но я не вижу значимого эффекта и влияния на рейтинг, единственная гипотеза которую стоит проверить, стандартная длительность поездки (менее 7 дней) отличается по рейтингу от любой другой длительности поездки (бинарный признак).

In [74]:
# Бинарный признак стандартной длительности поездки 1-7 ночей
data['standard_stay'] = (
    (data['nights_stayed'] > 0) & (data['nights_stayed'] < 7)
).astype('int8')

In [75]:
# Бинарные признаки
binary_features = ['is_mobile', 'king_size_bed', 'without_window', 'standard_stay']
binary_titles = [
    'Рейтинг: отзыв с мобильного устройства',
    'Рейтинг: наличие большой кровати', 
    'Рейтинг: номер без окна',
    'Рейтинг: стандартная длительность поездки'
]

fig = get_boxplots(binary_features, binary_titles, ncols=2, colors=True)
for i, ax in enumerate(fig.axes[:len(binary_features)]):
    ax.set_xticks([0, 1])
    ax.set_xticklabels(['Нет', 'Да'])

plt.tight_layout()
# Для отображения раскомментировать plt.show()
# plt.show()
plt.close()

<div align="center">
<image src="data/image/plt_9.png" width="800"></div>

In [76]:
# Категориальные признаки
cat_features = ['trip_type', 'travelers_type', 'room_type']
cat_titles = [
    'Рейтинг: тип поездки', 'Рейтинг: тип путешественников', 'Рейтинг: тип комнаты'
]

fig = get_boxplots(cat_features, cat_titles, ncols=3, figsize=(15, 5))
for i, ax in enumerate(fig.axes[:len(binary_features)]):
    ax.tick_params(axis='x', rotation=45, labelsize=9)

plt.tight_layout()
# Для отображения раскомментировать plt.show()
# plt.show()
plt.close()

<div align="center">
<image src="data/image/plt_10.png" height="450"></div>

Итого создано 8 новых признака, `is_mobile`, `trip_type`, `travelers_type,` `king_size_bed`, `without_window`, `nights_stayed`, `standard_stay`, `room_type`.
1. `is_mobile` - скорее всего бесполезен, нет никакой разницы в рейтинге;
2. `trip_type` - в целом медиана бизнес поездок значительно ниже, чем у поездок на отдых. Думаю признак полезен;
3. `travelers_type` - значимые различая есть у одиночных путешествий - самый низкий рейтинг, у путешествующих парой или с друзьями самый высокий рейтинг, остальные группы в целом можно было бы определить в одну, необходимо провести тест на статистическую значимость между группами, если есть необходимость переопределить;
4. `king_size_bed` и `without_window` показывают очевидное влияние на рейтинг, но надо понимать, что признаки распределены очень неравномерно. Всего в данных около 3000 отзывов, где указано, что номер без окна. Королевская кровать указана примерно в 15% отзывах. Повлияет ли это как-то на модель? Необходим тест на статистическую значимость;
5. Количество ночей (`nights_stayed`) самый странный признак, по графику и статистическим данным видно, что в целом основная масса наших отзывов это несколько ночей пребывания в отеле. Рейтинг для стандартной длительности отдыха 1-7 дней практически одинаков, далее начинает снижаться до 10 дней, а дальше не подчиняется какой либо разумной логике. Я пробовала определить в очевидные мне категории (1-3 ночи, 3-14, более 14 ночей), которые обозначила ранее, но слишком не равномерное распределение и рейтинг для таких групп практически одинаков. Пробовала посчитать по квантилям (для инфо 90% всех данных до 4 ночей, 99% до 8), но также не показывает никакой разницы. Более менее отличие имеет разница до недели отпуска и все остальное (`standard_stay`), при этом не понятна ценность признака, так как не равномерное распределение. Статистические тесты должны показать значимо ли отличие. Категории не стала оставлять сразу, там очевидное отсутствие зависимости;
6. `room_type` - текущие категории, определенные мной показывают разницу в рейтинге. Но с ним также есть пара нюансов, не уверена, что отнесла правильно определенный тег к нужной категории, точно уверено могу сказать, что максимальный рейтинг показали отели Luxury, минимальный рейтинг у номеров Guestroom. Но их довольно мало, чтобы выделять, например, в бинарный признак, поэтому распределила на группы интуитивно. Надо проверять, значим этот признак или нет. Есть ли статистическая разница между мной созданными группами.

In [77]:
# Удаляем сразу первоначальный признак
data.drop('tags', axis=1, inplace=True)

#### Анализ признаков `review_total_negative_word_counts`, `review_total_positive_word_counts`

In [78]:
# Разбиваю на категории по квантилям
data['negative_words_category'] = pd.qcut(
    data['review_total_negative_word_counts'],
    q=3, labels=['short', 'medium', 'long']
).astype('category')
data['positive_words_category'] = pd.qcut(
    data['review_total_positive_word_counts'],
    q=3, labels=['short', 'medium', 'long']
).astype('category')

# Визуализируем их
new_features = ['negative_words_category', 'positive_words_category']
titles = ['Распределение оценок по длине негативных отзывов',
          'Распределение оценок по длине позитивных отзывов']
get_boxplots(new_features, titles)

# Раскомментировать для отображения
# plt.show()
plt.close()

# Посмотрим статистику
get_stats('negative_words_category')
get_stats('positive_words_category')

Статистика reviewer_score по negative_words_category:


,count,mean,std,min,25%,50%,75%,max
negative_words_category,,,,,,,,
short,139243.0,9.20,1.14,2.5,8.8,9.6,10.0,10.0
medium,120393.0,8.32,1.52,2.5,7.5,8.8,9.6,10.0
long,126860.0,7.59,1.78,2.5,6.3,7.9,9.2,10.0


Статистика reviewer_score по positive_words_category:


,count,mean,std,min,25%,50%,75%,max
positive_words_category,,,,,,,,
short,136369.0,7.75,1.88,2.5,6.7,7.9,9.2,10.0
medium,121769.0,8.56,1.45,2.5,7.9,9.0,9.6,10.0
long,128358.0,8.92,1.25,2.5,8.3,9.2,10.0,10.0


<div align="center">
<image src="data/image/plt_11.png" height="400"></div>

Наблюдается четкая зависимость между длиной отзывов и оценкой: чем длиннее негативный отзыв, тем ниже рейтинг отеля, в то время как подробные позитивные отзывы соответствуют более высоким оценкам. Оба признака демонстрируют статистически значимое влияние на целевую переменную. 
Не понятно имело ли смысл определять их в категории, возможно стоит оставить в первоначальном виде (числовом формате), так как очевидно признаки показывают строгую линейную зависимость. Необходимо сравнить корреляцию с целевой переменной новых созданных признаков и первоначальные признаки, для выбора, какие признаки оставить, так как новые признаки очевидно мультиколлинеарны с изначальными признаками.

### Признаки отеля - `total_number_of_reviews` и `additional_number_of_scoring`, `average_score`

In [79]:
# Большое количество отзывов отеля отличается от малого количества отзывов
# Категории по количеству отзывов
data['review_count_category'] = pd.qcut(
    data['total_number_of_reviews'], q=4,
    labels=['Мало', 'Умеренно', 'Много', 'Oчень много']
)

# Большое количество молчаливых оценок оказывает влияние на рейтинг по сравнению с малым
# Категории по количеству оценок без отзывов
data['score_count_category'] = pd.qcut(
    data['additional_number_of_scoring'],
    q=4, 
    labels=['Мало', 'Умеренно', 'Много', 'Oчень много']
)

# Посмотрим статистику
get_stats('review_count_category')
get_stats('score_count_category')

Статистика reviewer_score по review_count_category:


,count,mean,std,min,25%,50%,75%,max
review_count_category,,,,,,,,
Мало,97013.0,8.57,1.57,2.5,7.9,9.2,10.0,10.0
Умеренно,96590.0,8.43,1.60,2.5,7.5,8.8,9.6,10.0
Много,96905.0,8.32,1.65,2.5,7.5,8.8,9.6,10.0
Oчень много,95988.0,8.27,1.70,2.5,7.5,8.8,9.6,10.0


Статистика reviewer_score по score_count_category:


,count,mean,std,min,25%,50%,75%,max
score_count_category,,,,,,,,
Мало,97076.0,8.52,1.59,2.5,7.9,9.2,9.6,10.0
Умеренно,96650.0,8.42,1.61,2.5,7.5,8.8,9.6,10.0
Много,96271.0,8.33,1.66,2.5,7.5,8.8,9.6,10.0
Oчень много,96499.0,8.32,1.68,2.5,7.5,8.8,9.6,10.0


Не вижу значительной разницы в рейтинге, есть небольшой скачок рейтинга для категорий - "мало", в обоих признаках, возможно в данную категорию попали новые отели, которым физически не успели поставить много оценок и написать много отзывов, и у них выше рейтинг? Создам бинарный признак для предполагаемых новых отелей.

In [80]:
# Создаем бинарный признак для потенциально новых отелей
# которые имеют мало отзывов И мало оценок
data['possibly_new_hotel'] = (
    (data['review_count_category'] == 'Мало') & 
    (data['score_count_category'] == 'Мало')
).astype('int8')

# Посмотрим статистику
get_stats('possibly_new_hotel')

Статистика reviewer_score по possibly_new_hotel:


,count,mean,std,min,25%,50%,75%,max
possibly_new_hotel,,,,,,,,
0,313248.0,8.36,1.65,2.5,7.5,8.8,9.6,10.0
1,73248.0,8.58,1.57,2.5,7.9,9.2,10.0,10.0


In [81]:
new_features = ['review_count_category', 'score_count_category', 'possibly_new_hotel']
titles = [
    'Рейтинг vs Количество отзывов',
    'Рейтинг vs Оценки без отзыва',
    'Вероятно новый отель']
fig = get_boxplots(new_features, titles, colors=True)

fig.axes[2].set_xticks([1, 0])
fig.axes[2].set_xticklabels(['Да', 'Нет'])

plt.tight_layout()
# Раскомментировать для отображения
# plt.show()
plt.close()

<div align="center">
<image src="data/image/plt_12.png" height="400"></div>

Разница прослеживается только для категории мало отзывов/мало оценок. Необходимо провести тесты и решить имеется ли статистически значимая разница между новыми категориями признаков, кроме того проверить корреляцию с целевой переменной, для решения какие из признаков оставить для модели - новые или же просто оставить первоначальные, и модель сама найдет взаимосвязь?

Проверим гипотезу: доля молчаливых отзывов оказывает влияние на рейтинг (чем меньше доля тем честнее оценка, и возможно ниже):

In [82]:
# Доля молчаливых оценок
data['silent_score_ratio'] = (
    data['additional_number_of_scoring'] / data['total_number_of_reviews']
).astype('float32')
data['silent_score_ratio'].describe()

count    515431.000000
mean          0.179090
std           0.075778
min           0.020408
25%           0.109046
50%           0.176425
75%           0.239165
max           0.397059
Name: silent_score_ratio, dtype: float64

In [83]:
# Создаем категории и визуализирую их на графике
data['silent_ratio_category'] = pd.qcut(
    data['silent_score_ratio'], q=3,
    labels=['Низкая', 'Средняя', 'Высокая']
).astype('category')
get_boxplots(
    ['silent_ratio_category'],
    ['Рейтинг по категориям доли молчаливых оценок'],
    ['Категория доли молчаливых оценок']
)

# Раскомментировать для отображения
# plt.show()
plt.close()

# Посмотрим статистику
get_stats('silent_ratio_category')

Статистика reviewer_score по silent_ratio_category:


,count,mean,std,min,25%,50%,75%,max
silent_ratio_category,,,,,,,,
Низкая,129037.0,8.38,1.62,2.5,7.5,8.8,9.6,10.0
Средняя,129397.0,8.36,1.67,2.5,7.5,8.8,9.6,10.0
Высокая,128062.0,8.45,1.61,2.5,7.5,8.8,9.6,10.0


<div align="center">
<image src="data/image/plt_13.png" height="400"></div>

Четкой закономерности между долей молчаливых оценок и рейтингом не обнаружено. Логично было бы предположить, что меньшая доля "молчаливых" оценок соответствует более честным отзывам и, возможно, более низкому среднему рейтингу. Однако данные показывают противоречивую картину: при низкой доле молчаливых отзывов рейтинг ниже среднего, затем наблюдается скачок для категории высокая. После проведения тестов приму решение, что делать с признаками.

Теперь проверим: влияет ли популярность отеля на рейтинг отеля:

In [84]:
# Создаю категориальный признак
data['average_score_category'] = pd.qcut(
    data['average_score'], q=3,
    labels=['Низкая оценка', 'Средняя оценка', 'Высокая оценка']
).astype('category')
get_stats('average_score_category')

Статистика reviewer_score по average_score_category:


,count,mean,std,min,25%,50%,75%,max
average_score_category,,,,,,,,
Низкая оценка,142342.0,7.78,1.80,2.5,6.7,7.9,9.2,10.0
Средняя оценка,140103.0,8.52,1.51,2.5,7.5,8.8,9.6,10.0
Высокая оценка,104051.0,9.08,1.20,2.5,8.8,9.6,10.0,10.0


Результат очевидный, чем выше оценка отеля, тем выше средний рейтинг отеля, так как в целом оценка отеля определена по рейтингу отелей в прошлом, то по сути описывают одинаковый признак, соответственно скорее всего сам по себе признак средняя оценка отеля сильно коррелирует с рейтингом (необходима проверка на мультиколлинеарность), возможно определенные категории в таком случае лучший выбор чем первоначальный признак, исключат сильную корреляцию с целевой,при этом будут маячком, что данные отели в прошлом показывали более высокие оценки. Но опять таки в рамках настоящего задания это улучшит метрику, а в рамках реальной задачи, где модель столкнется с отелями не из этой выборки данный признак только ее запутает.

## Отбор признаков

### Статистические тесты

#### Подготовка

Более точная формулировка гипотез после создания признаков. Гипотезы разделю по типу данных а не тому, что они описываю, для удобства дальнейшей проверки.

Так как в нашем случае полезно, как уменьшение рейтинга так и увеличение, для бинарных признаков общая формулировка гипотезы означает любое отличие рейтинга между двумя проверяемыми группами. Перечень новых бинарных признаков и альтернативные гипотезы для них:

- chain_hotel - имеется разница между медианным рейтингом у сетевых и не сетевых отелей;
- is_tourist - имеется значимая разница между туристами из других стран и резидентами;
- expert_reviewer - имеется разница в рейтинге между отзывами оставленными опытными пользователями;
- has_negative - наличие негативного отзыва влияет на оценку;
- bad_room - упоминание комнаты в негативном отзыве влияет на оценку;
- bad_service - упоминание сервиса в негативном отзыве влияет на оценку;
- bad_food - упоминание еды в негативном отзыве влияет на оценку;
- bad_price - упоминание цены в негативном отзыве влияет на оценку;
- bad_location - упоминание локации в негативном отзыве влияет на оценку;
- bad_option - упоминание опций отеля в негативном отзыве влияет на оценку;
- has_positive - наличие позитивного отзыва влияет на оценку;
- good_location - упоминание локации в позитивном отзыве влияет на оценку;
- good_staff - упоминание персонала в позитивном отзыве влияет на оценку;
- good_clean - упоминание чистоты в позитивном отзыве влияет на оценку;
- good_food - упоминание еды в позитивном отзыве влияет на оценку;
- good_room - упоминание комнаты в позитивном отзыве влияет на оценку;
- good_overall - упоминание слов восхищения в позитивном отзыве влияет на оценку;
- is_mobile - наличие тега, что отзыв оставлен с мобильного влияет на оценку;
- king_size_bed - наличие тега, о большой кровати влияет на оценку;
- without_window - наличие тега, о номере без окна влияет на оценку;
- standard_stay - на оценку влияет стандартное количество ночей пребывания (< 7 дней);
- possibly_new_hotel - вероятно новые отели показывают отличие рейтинга в сравнении с остальными;

Для категориальных признаков будет проведено 2 проверки, 1 проверка есть ли в целом отличаи в медианных рейтингах внутри категорий в признаке и отдельно проведу тесту между различными категориями в признаках если они пройдут первый тест. Итак, гипотезы следующие:

1. По категории в целом:
review_year - год отзыва влияет на рейтинг;
review_month - месяц отзыва влияет на рейтинг;
review_quarter - квартал отзыва влияет на рейтинг;
since_review - давность отзыва влияет на рейтинг; 
country - страна расположения отеля влияет на рейтинг;
dist_center_cat - категория расположения отеля относительно центра влияет на рейтинг;
dist_airport_cat - категория расположения отеля относительно аэропорта влияет на рейтинг;
region_nationality - регион национальности гостя отеля влияет на рейтинг;
reviewer_experience - категория опыта пользователя booking влияет на рейтинг;
trip_type - тип поездки (бизнес, отдых, другое) влияет на рейтинг;
travelers_type - тип путешественника (пара, семья и тп.) влияет на рейтинг;
room_type - категория типа комнаты влияет на рейтинг;
negative_words_category - категория длины негативного отзыва влияет на рейтинг;
positive_words_category - категория длины позитивного отзыва влияет на рейтинг;
review_count_category - категория количества отзывов об отеле влияет на рейтинг;
score_count_category - категория количества оценок без отзыва об отеле влияет на рейтинг;
silent_ratio_category - категория доли молчаливых отзывов об отеле влияет на рейтинг;
average_score_category - категория популярности отеля по средним оценкам влияет на рейтинг.

2. Сравнение категорий внутри признака:
review_year - разница в рейтинге есть между всеми годами отзыва;
review_month - разница в рейтинге есть между всеми месяцами отзыва;
review_quarter - разница в рейтинге есть между всеми кварталами отзыва;
since_review - разница в рейтинге есть между всеми категориями давности отзыва;
country - разница в рейтинге есть между странами расположения отеля;
dist_center_cat - разница в рейтинге есть между всеми категориями удаленности от центра;
dist_airport_cat - разница в рейтинге есть между всеми категориями удаленности от центра (основная теория, что максимально близкие возможно равны максимально дальним);
region_nationality - разница в рейтинге есть между регионами национальности гостя;
reviewer_experience - разница в рейтинге есть между всеми категориями опыта пользователя booking;
trip_type - разница в рейтинге есть между всеми категориями тип поездки (бизнес, отдых, другое);
travelers_type - разница в рейтинге есть между всеми категориями типа путешественника (пара, семья и тп.);
room_type - разница в рейтинге есть между всеми категориями типа комнаты;
negative_words_category - разница в рейтинге есть между всеми категориями длины негативного отзыва;
positive_words_category - разница в рейтинге есть между всеми категориями длины позитивного отзыва;
review_count_category - разница в рейтинге есть между всеми категориями количества отзывов об отеле;
score_count_category - разница в рейтинге есть между всеми категориями количества оценок без отзыва об отеле;
silent_ratio_category - разница в рейтинге есть между всеми категориями доли молчаливых отзывов об отеле;
average_score_category - разница в рейтинге есть между всеми категориями популярности отеля по средним оценкам.

План по очистке следующий, если после первой проверки признака мы не можем отвергнуть нулевую гипотезу, то просто избавляемся от признака. После второй проверки, между категориями внутри признака, мы принимаем решение - переопределить какие-то категории в другие (объединить некоторые категории в одну), удалить признак (если логики не показывает и есть, например, числовой признак описывающий данный признак), или возможно выделить бинарный признак если есть разница только для одной категории внутри признака.

#### Бинарные

In [85]:
# Задаю список бинарных признаков
binary_features = [
    'chain_hotel', 'is_tourist', 'expert_reviewer', 'has_negative', 'bad_room',
    'bad_service', 'bad_food', 'bad_price', 'bad_location', 'bad_option',
    'has_positive', 'good_location', 'good_staff', 'good_clean', 'good_food',
    'good_room', 'good_overall', 'is_mobile', 'king_size_bed', 'without_window',
    'standard_stay', 'possibly_new_hotel', 'recent_review'
]

**1. Проверка различия медиан рейтинга в бинарных признаках**

- $H_0$: Медианные рейтинги для значения 1 и 0 одинаковы

     $H_0: m_1 = m_2$

- $H_0$: Медианные рейтинги для значения 1 и 0 различаются

     $H_0: m_1 \ne  m_2$

In [86]:
binary_test_norm(binary_features)

Нормально распределенных данных нет


In [87]:
recommend_test(2,0,0)

'Рекомендуемый тест: U-критерий Манна-Уитни: scipy.stats.mannwhitneyu'

In [88]:
pairwise_u_test(binary_features, is_binary=True)


chain_hotel: Отвергаем нулевую гипотезу

is_tourist: Отвергаем нулевую гипотезу

expert_reviewer: Отвергаем нулевую гипотезу

has_negative: Отвергаем нулевую гипотезу

bad_room: Отвергаем нулевую гипотезу

bad_service: Отвергаем нулевую гипотезу

bad_food: Отвергаем нулевую гипотезу

bad_price: Отвергаем нулевую гипотезу

bad_location: Отвергаем нулевую гипотезу

bad_option: Отвергаем нулевую гипотезу

has_positive: Отвергаем нулевую гипотезу

good_location
1 и 0: p-value=0.889 > 0.050. У нас нет оснований отвергнуть нулевую гипотезу.

good_staff: Отвергаем нулевую гипотезу

good_clean: Отвергаем нулевую гипотезу

good_food: Отвергаем нулевую гипотезу

good_room: Отвергаем нулевую гипотезу

good_overall: Отвергаем нулевую гипотезу

is_mobile: Отвергаем нулевую гипотезу

king_size_bed: Отвергаем нулевую гипотезу

without_window: Отвергаем нулевую гипотезу

standard_stay: Отвергаем нулевую гипотезу

possibly_new_hotel: Отвергаем нулевую гипотезу

recent_review: Отвергаем нулевую гипотез

**Вывод:** Не подтвердилась гипотеза для признака 'good_location', не показывает статистически значимых различий (как мы и предполагали в предварительном анализе). Признак не нужен удаляю его.

In [89]:
data.drop(['good_location'], axis=1, inplace=True)

In [90]:
# Обновленный список бинарных признаков
binary_features = [
    'chain_hotel', 'is_tourist', 'expert_reviewer', 'has_negative', 'bad_room',
    'bad_service', 'bad_food', 'bad_price', 'bad_location', 'bad_option',
    'has_positive', 'good_staff', 'good_clean', 'good_food',
    'good_room', 'good_overall', 'is_mobile', 'king_size_bed', 'without_window',
    'standard_stay', 'possibly_new_hotel', 'recent_review'
]
exclude_cols = ['sample']
cat_cols = [col for col in data.select_dtypes(include=['category']).columns 
           if col not in exclude_cols]

num_cols = [col for col in data.select_dtypes(include='number').columns 
           if col not in binary_features and col != 'reviewer_score']

#### Категориальные в целом

**2. Гипотезы для категориальных признаков:**
- $H_0$: Медианные рейтинги одинаковы во всех категориях
    
     $H_0: m_1 = m_2 = ... = m_k$

- $H_1$: Хотя бы в одной паре категорий медианные рейтинги различаются

     $H_1: \exists i,j : m_i \ne m_j$

In [91]:
categorical_test_norm(cat_cols)

Нормально распределенных данных нет


In [92]:
recommend_test(3,0,0)

'Рекомендуемый тест: Критерий Краскела-Уоллиса: scipy.stats.kruskal'

In [93]:
kruskal_test(cat_cols)

Для признака review_year: отвергаем нулевую гипотезу (p-value:0.000<=0.05)
Для признака review_month: отвергаем нулевую гипотезу (p-value:0.000<=0.05)
Для признака review_quarter: отвергаем нулевую гипотезу (p-value:0.000<=0.05)
Для признака since_review: отвергаем нулевую гипотезу (p-value:0.000<=0.05)
Для признака country: отвергаем нулевую гипотезу (p-value:0.000<=0.05)
Для признака dist_center_cat: отвергаем нулевую гипотезу (p-value:0.000<=0.05)
Для признака dist_airport_cat: отвергаем нулевую гипотезу (p-value:0.000<=0.05)
Для признака region_nationality: отвергаем нулевую гипотезу (p-value:0.000<=0.05)
Для признака reviewer_experience: отвергаем нулевую гипотезу (p-value:0.000<=0.05)
Для признака trip_type: отвергаем нулевую гипотезу (p-value:0.000<=0.05)
Для признака travelers_type: отвергаем нулевую гипотезу (p-value:0.000<=0.05)
Для признака room_type: отвергаем нулевую гипотезу (p-value:0.000<=0.05)
Для признака negative_words_category: отвергаем нулевую гипотезу (p-value:0.

Все признаки показали отличая внутри категорий, соответственно удалять пока ничего не будем, проверим различая между конкретными категориями:

#### Категориальные внутри категорий

**3. Внутри категорий.** Предыдущий тест показал, что во всех признаках есть статистически значимые различия между некоторыми категориями. Однако, учитывая, что категории задавались вручную и некоторые из них могут быть избыточными, я провела попарный тест категорий внутри каждого категориального признака.

**Критерии для оптимизации признаков:**
1. Объединение - если неразличимые группы можно логически объединить;
2. Удаление - если категоризация числового признака не показывает различий;
3. Выбор - если несколько признаков описывают один тип данных, оставить наиболее информативный;
4. Бинаризация - если только одна пара показывает различие;

- $H_0$: группы внутри признака равны (медианы одинаковы)

- $H_1$: группы внутри признака не равны (медианы различаются)

На нормальность данные уже проверены, все группы независимы друг от друга, поэтому используем тест Манна-Уитни.

In [94]:
recommend_test(2,0,0)

'Рекомендуемый тест: U-критерий Манна-Уитни: scipy.stats.mannwhitneyu'

In [95]:
pairwise_u_test(cat_cols, bonferroni=True)


review_year
2016 и 2017: p-value=0.272 > 0.017. У нас нет оснований отвергнуть нулевую гипотезу.

review_month
2 и 1: p-value=0.001 > 0.001. У нас нет оснований отвергнуть нулевую гипотезу.
1 и 3: p-value=0.332 > 0.001. У нас нет оснований отвергнуть нулевую гипотезу.
9 и 11: p-value=0.028 > 0.001. У нас нет оснований отвергнуть нулевую гипотезу.
9 и 7: p-value=0.001 > 0.001. У нас нет оснований отвергнуть нулевую гипотезу.
3 и 12: p-value=0.010 > 0.001. У нас нет оснований отвергнуть нулевую гипотезу.
12 и 4: p-value=0.129 > 0.001. У нас нет оснований отвергнуть нулевую гипотезу.
12 и 8: p-value=0.022 > 0.001. У нас нет оснований отвергнуть нулевую гипотезу.
5 и 6: p-value=0.720 > 0.001. У нас нет оснований отвергнуть нулевую гипотезу.
5 и 4: p-value=0.003 > 0.001. У нас нет оснований отвергнуть нулевую гипотезу.
5 и 8: p-value=0.024 > 0.001. У нас нет оснований отвергнуть нулевую гипотезу.
11 и 7: p-value=0.357 > 0.001. У нас нет оснований отвергнуть нулевую гипотезу.
6 и 4: p-value

Выводы и новые гипотезы, для данных которые не смогли подтвердить.
1. review_year отсутствует различие в паре 2016 и 2017 год. Тогда новая гипотеза 2015 год относительно всех других годов дает влияние на рейтинг (создаем бинарный признак);
2. Большое количество месяцев не имеют между собой различий в рейтинге. Так как данный признак описывает туже зависимость, что и квартал отзыва, считаю, что его можно удалить полностью.
3. 4 и 3 квартал в данных не показывает отличий. Что в целом повторяет наблюдения сделанные при анализе признака, так как наличие отдельных признаков 3 квартала и 4 квартала для модели явно избыточно, объединим их;
4. country - для стран, которые имеют одинаковый средний рейтинг также проведем объединение, это уменьшит количество признаков после кодирования, данные не будут утеряны, смысл останется;
5. 'dist_airport_cat' - ожидаемый результат, который подтвердит мою гипотезу, что для определенных целей отель расположенный очень близко к отелю более высоко ценится, чем более удаленные. Я могла бы категорию "очень близко" определить в бинарный признак, однако не вижу в этом смыла, так как потеряется суть. Считаю, что для модели полезно передать данные в виде порядкового кодирования, где 0 - "очень близко", а далее по увеличению расстояния рейтинг с самого низкого растет до уровня "самый дальний";
6. 'region_nationality' - "Asia" и "Africa" статистически неразличимы в данных, так как Африки в данных очень малое количество определю их в "Other", где судя по всему уже есть похожие по рейтингу страны.
7. 'travelers_type' - "Couple" и "Travelers with friends" объединю в одну группу, "Travelers with friends" малочисленна, кроме того часто подразумевает одинаковый отдых с парой (например, две подруги - стандартный номер и т.п.);
8. Для признака 'score_count_category' категории "много" и "очень много" объединю в один признак "много".
9. Для признака 'silent_ratio_category' не показал разницы в тесте между категориями "низкая" и "средняя доли". Так как в данном признаке всего 3 категории, оставшуюся высокую долю, которая показывает разницу выделю в бинарный признак.

In [96]:
# 1. Создаем бинарный признак отзывы 2015 года
data['review_2015'] = data['review_year'].apply(
    lambda x: 1 if x == 2015 else 0
).astype('int8')

# 3. Объединяю 3 и 4 квартал в признаке кварталов
data['review_quarter'] = data['review_quarter'].apply(
    lambda x: '3_4' if  x in [3, 4] else x
).astype('category')

# 4. Переопределяю страны
country_type = {
    'United Kingdom': 'UK_IT',
    'Italy': 'UK_IT',
    'France': 'FR_NL',
    'Netherlands': 'FR_NL',
    'Austria': 'AT',
    'Spain': 'ES'
}
data['country'] = data['country'].map(country_type).astype('category')

# 6. Переопределяем Африку в другие
data['region_nationality'] = data['region_nationality'].apply(
    lambda x: 'Other' if x == 'Africa' else x
).astype('category')

# 7. Объединяю 'Couple' и 'Travelers with friends'
data['travelers_type'] = data['travelers_type'].apply(
    lambda x: 'Couple_Friends' if x in ['Couple', 'Travelers with friends'] else x
).astype('category')

# 8. Объединяю 'Много' и 'Очень много'
data['score_count_category'] = data['score_count_category'].apply(
    lambda x: 'Много' if x == 'Oчень много' else x
).astype('category')

# 9. Создаем бинарный признак высокая доля молчаливых отзывов
data['hi_silent_ratio'] = data['silent_ratio_category'].apply(
    lambda x: 1 if x == 'Высокая' else 0
).astype('int8')


# Удаляю не нужные отработанные признаки
drop_list = ['review_year', 'review_month', 'silent_ratio_category']
data.drop(drop_list, axis=1, inplace=True)

Проверяем новые гипотезы:

- Для бинарных -  'review_2015', 'hi_silent_ratio' на различие между парами
- Для категориальных - 'seasons', 'review_quarter', 'country', 'region_nationality', 'travelers_type', 'score_count_category' на различие между группами категорий в признаке.

In [97]:
# Проверка нормальности
binary_test_norm(['review_2015', 'hi_silent_ratio'])
categorical_test_norm(
    ['review_quarter', 'country', 'region_nationality', 
     'travelers_type', 'score_count_category']
)

Нормально распределенных данных нет
Нормально распределенных данных нет


In [98]:
# Проведение тестов
pairwise_u_test(['review_2015', 'hi_silent_ratio'], is_binary=True)
pairwise_u_test(
    ['review_quarter', 'country', 'region_nationality', 
     'travelers_type', 'score_count_category'],bonferroni=True
)


review_2015: Отвергаем нулевую гипотезу

hi_silent_ratio: Отвергаем нулевую гипотезу
Во всех признаках все группы статистически различаются

review_quarter: Отвергаем нулевую гипотезу

country: Отвергаем нулевую гипотезу

region_nationality: Отвергаем нулевую гипотезу

travelers_type: Отвергаем нулевую гипотезу

score_count_category: Отвергаем нулевую гипотезу
Во всех признаках все группы статистически различаются


Таким образом, все статистически значимые различия мы обработали, все признаки показывают различие в рейтинге, можно двигаться дальше, проверять корреляцию, проходить анова тесты и хи-квадрат, для выявления наиболее значимых признаков. Для дальнейших тестов необходимо сначала закодировать категориальные признаки

In [99]:
# Обновляю трейн
train_data = data[train_mask].copy()
# Обновляю колонки
exclude_cols = ['sample']
cat_cols = [col for col in data.select_dtypes(include=['category']).columns 
           if col not in exclude_cols]

### Кодирование признаков

In [100]:
# Порядковое кодирование
# Порядковые признаки - задаем порядок вручную
ordinal_mapping = {
    'since_review': ['3_months', '1_year', 'old'],
    'dist_center_cat': ['Очень близко', 'Близко', 'Далеко', 'Очень далеко'],
    'dist_airport_cat': ['Очень близко', 'Близко', 'Далеко', 'Очень далеко'],
    'reviewer_experience': ['Новый пользователь', 'Активный пользователь', 'Эксперт'],
    'negative_words_category': ['short', 'medium', 'long'],
    'positive_words_category': ['short', 'medium', 'long'],
    'review_count_category': ['Мало', 'Умеренно', 'Много', 'Oчень много'],
    'score_count_category': ['Мало', 'Умеренно', 'Много'],
    'average_score_category': ['Низкая оценка', 'Средняя оценка', 'Высокая оценка']
}
for col, order in ordinal_mapping.items():
    data[col] = pd.Categorical(
        data[col], categories=order, ordered=True
).codes.astype('int8')

In [101]:
# One-Hot для номинальных с малым числом категорий (<10)
onehot_columns = [
    'review_quarter', 'country', 'region_nationality', 'trip_type',
    'room_type', 'travelers_type'
]
encoder = ce.OneHotEncoder(cols=onehot_columns, use_cat_names=True)
data = encoder.fit_transform(data)
onehot_cols = [col for col in data.columns 
               if any(prefix in col for prefix in onehot_columns)]
data[onehot_cols] = data[onehot_cols].astype('int8')

### Корреляция

Проверить сразу все признаки Одновременно довольно затруднительно. План такой: у меня много категориальных признаков, созданных из других числовых признаков, они почти 100% между собой коррелируют, соответственно избыточны для модели, проверяю их первыми. Отдельно проверю на корреляцию признаки времени и местоположения (объединю в одну карту). И отдельно проверю признаки созданные из отзывов и тегов, они возможно коррелируют между собой. Кроме того есть разные признаки описывающий одни и те же данные. Проверю между ними корреляцию, и оставлю только те признаки, которые показывают связь с целевой переменной больше.

In [102]:
# Списки для проверки на корреляцию
check_list_1 = [
    'reviewer_score', 'days_since_review', 'since_review', 'recent_review',
    'total_number_of_reviews_reviewer_has_given', 'reviewer_experience', 'expert_reviewer',
    'has_negative', 'review_total_negative_word_counts', 'negative_words_category',
    'has_positive', 'review_total_positive_word_counts', 'positive_words_category', 
    'good_overall', 'nights_stayed', 'standard_stay', 'total_number_of_reviews',
    'review_count_category', 'possibly_new_hotel', 'additional_number_of_scoring',
    'score_count_category', 'silent_score_ratio', 'average_score_category',
    'average_score'
]
check_list_2 = [
    'reviewer_score', 'review_2015', 'review_quarter_3_4', 'review_quarter_2',
    'review_quarter_1', 'region_nationality_Europe', 'region_nationality_Americas',
    'region_nationality_Other', 'region_nationality_Oceania',
    'region_nationality_Asia','is_tourist', 'country_UK_IT', 'country_FR_NL',
    'country_ES', 'country_AT', 'dist_center', 'dist_center_cat', 'dist_airport',
    'dist_airport_cat', 'chain_hotel'
]
check_list_3 = [
    'reviewer_score', 'has_negative', 'bad_room', 'bad_service', 'bad_food', 'bad_price',
    'bad_location', 'bad_option', 'has_positive', 'good_staff', 'good_clean',
    'good_food', 'good_room', 'good_overall', 'is_mobile', 'king_size_bed',
    'without_window', 'trip_type_Leisure trip', 'trip_type_Business trip',
    'trip_type_other',  'travelers_type_Couple_Friends', 'travelers_type_Group',
    'travelers_type_Solo traveler', 'travelers_type_Family with young children',
    'travelers_type_Family with older children', 'room_type_Стандарт',
    'room_type_Люкс', 'room_type_Эконом', 'nights_stayed', 'standard_stay'
]

# Считаем по Спирмену корреляцию по трейн данным
train_data = data[train_mask]
corr_spearman_1 = train_data[check_list_1].corr(method = 'spearman')
corr_spearman_2 = train_data[check_list_2].corr(method = 'spearman')
corr_spearman_3 = train_data[check_list_3].corr(method = 'spearman')

# Создаем три тепловых карты
fig, axes = plt.subplots(3, 1, figsize=(20, 40))

mask_1 = abs(corr_spearman_1) < 0.2
mask_2 = abs(corr_spearman_2) < 0.2
mask_3 = abs(corr_spearman_3) < 0.2

# Первая heatmap
sns.heatmap(
    corr_spearman_1, annot=True, mask=mask_1, fmt='.2f', square=True,
    center=0, linewidths=0.5, linecolor='white', ax=axes[0]
)
axes[0].set_title('Матрица корреляции 1 - Основные признаки')
axes[0].tick_params(axis='x', rotation=75)
axes[0].tick_params(axis='y', rotation=0)

# Вторая heatmap  
sns.heatmap(
    corr_spearman_2, annot=True, mask=mask_2, fmt='.2f', square=True,
    center=0, linewidths=0.5, linecolor='white', ax=axes[1]
)
axes[1].set_title('Матрица корреляции 2 - География и время')
axes[1].tick_params(axis='x', rotation=45)
axes[1].tick_params(axis='y', rotation=0)

# Третья heatmap
sns.heatmap(
    corr_spearman_3, annot=True, mask=mask_3, fmt='.2f', square=True,
    center=0, linewidths=0.5, linecolor='white', ax=axes[2]
)
axes[2].set_title('Матрица корреляции 3 - Детали отзывов')
axes[2].tick_params(axis='x', rotation=75)
axes[2].tick_params(axis='y', rotation=0)

plt.tight_layout()
# Раскомментировать для отображения
# plt.show()
plt.close()

<div align="center">
<image src="data/image/plt_14.png" width="800"></div>

In [103]:
# Функция, которая поможет сравнить корреляцию, и удалить лишний признак
def choose_one(col_1, col_2, df=None, target='reviewer_score'):
    """Удаляет один из двух коррелирующих признаков на основе корреляции с целевой
    переменной.
    Args:
        col_1 (str): название первого признака
        col_2 (str): название второго признака  
        df (DataFrame): DataFrame с данными, по умолчанию data
        target (str): целевая переменная, по умолчанию 'reviewer_score'
    Returns:
        None: удаляет менее информативный признак из DataFrame
    """
    if df is None:
        df = data
    # Вычисляем корреляции
    corr_1 = abs(df[col_1].corr(df[target], method='spearman'))
    corr_2 = abs(df[col_2].corr(df[target], method='spearman'))
    # Оставляем, ту что больше
    if corr_1 > corr_2:
        print(f'Остается {col_1} (корреляция: {corr_1:.3f} > {corr_2:.3f})')
        df.drop(col_2, axis=1, inplace=True)
    else:
        print(f'Остается {col_2} (корреляция: {corr_2:.3f} > {corr_1:.3f})')
        df.drop(col_1, axis=1, inplace=True)

In [104]:
# Пары на первой матрице корреляций > 0.9
choose_one('days_since_review', 'since_review')
choose_one('total_number_of_reviews_reviewer_has_given', 'reviewer_experience')
choose_one('review_count_category', 'total_number_of_reviews')
choose_one('review_total_negative_word_counts', 'negative_words_category')
choose_one('review_total_positive_word_counts', 'positive_words_category')
choose_one('score_count_category', 'additional_number_of_scoring')
choose_one('average_score_category', 'average_score')
# Пары на второй матрице корреляций > 0.9
choose_one('dist_center', 'dist_center_cat')
choose_one('dist_airport', 'dist_airport_cat')

Остается days_since_review (корреляция: 0.003 > 0.001)
Остается reviewer_experience (корреляция: 0.015 > 0.014)
Остается total_number_of_reviews (корреляция: 0.042 > 0.038)
Остается review_total_negative_word_counts (корреляция: 0.265 > 0.250)
Остается review_total_positive_word_counts (корреляция: 0.177 > 0.160)
Остается additional_number_of_scoring (корреляция: 0.027 > 0.025)
Остается average_score (корреляция: 0.200 > 0.183)
Остается dist_center_cat (корреляция: 0.057 > 0.055)
Остается dist_airport_cat (корреляция: 0.009 > 0.006)


### Статистические групповые тесты

In [105]:
# Обновлю данные о признаках и типах, чтобы не запутаться
binary_cols = [
    'chain_hotel', 'is_tourist', 'expert_reviewer', 'has_negative', 'bad_room',
    'bad_service', 'bad_food', 'bad_price', 'bad_location', 'bad_option',
    'has_positive', 'good_staff', 'good_clean', 'good_food',
    'good_room', 'good_overall', 'is_mobile', 'king_size_bed', 'without_window',
    'standard_stay', 'possibly_new_hotel', 'review_2015', 'hi_silent_ratio',
    'recent_review'
]
num_cols = [
    'additional_number_of_scoring', 'average_score', 'review_total_negative_word_counts',
    'total_number_of_reviews', 'review_total_positive_word_counts', 'days_since_review',
    'nights_stayed', 'silent_score_ratio'
]

exclude_cols = ['sample', 'reviewer_score']
cat_cols = [
    col for col in data.columns 
    if col not in num_cols
    and col not in exclude_cols 
    and col not in binary_cols
]

# Обновляю трейн
train_data = data[train_mask]

Визуализируем отдельно какие признаки наиболее важны для модели, для числовых признаков используем f_classif ANOVA тест, для категориальных (и бинарных) использую Хи-квадрат

In [106]:
# 1. ANOVA для числовых признаков
y = train_data['reviewer_score']
imp_num = pd.Series(f_classif(train_data[num_cols], y)[0], index=num_cols)
imp_num.sort_values(inplace=True)

plt.figure(figsize=(8, 5))
sns.barplot(x=imp_num.values, y=imp_num.index, hue=imp_num.values, legend=False)
plt.title('Значимость числовых признаков (ANOVA)')
plt.xlabel('F-статистика')
plt.ylabel('Признак')
plt.gca().tick_params(axis='y', which='major', labelright=True, labelleft=False)
plt.tight_layout()
# Раскомментировать для отображения
# plt.show()
plt.close()

<div align="center">
<image src="data/image/plt_15.png" height="400"></div>

In [107]:
# 2. Хи-квадрат для категориальных признаков
# Преобразуем y в категорию? для хи-квадрат
y = train_data['reviewer_score'].astype(int)
# Хи-квадрат для категориальных
chi2_scores, p_values = chi2(train_data[cat_cols], y)
# Создаем Series для визуализации
imp_cat = pd.Series(chi2_scores, index=cat_cols)
imp_cat.sort_values(inplace=True)

# Визуализируем
plt.figure(figsize=(10, 5))
sns.barplot(x=imp_cat.values, y=imp_cat.index, hue=imp_cat.values, legend=False)
plt.title('Значимость категориальных признаков (хи-квадрат)')
plt.xlabel('Хи-квадрат статистика')
plt.ylabel('Признак')
plt.gca().tick_params(axis='y', which='major', labelright=True, labelleft=False)
plt.tight_layout()
# Раскомментировать для отображения
# plt.show()
plt.close()

<div align="center">
<image src="data/image/plt_16.png" height="400"></div>

In [108]:
# 3. Хи-квадрат для бинарных признаков
chi2_scores_bin, p_values_bin = chi2(train_data[binary_cols], y)

imp_bin = pd.Series(chi2_scores_bin, index=binary_cols)
imp_bin.sort_values(inplace=True)

# Визуализируем
plt.figure(figsize=(8, 5))
sns.barplot(x=imp_bin.values, y=imp_bin.index, hue=imp_bin.values, legend=False)
plt.title('Значимость категориальных признаков (хи-квадрат)')
plt.xlabel('Хи-квадрат статистика')
plt.ylabel('Признак')
plt.gca().tick_params(axis='y', which='major', labelright=True, labelleft=False)
plt.tight_layout()
# Раскомментировать для отображения
# plt.show()
plt.close()

<div align="center">
<image src="data/image/plt_17.png" height="400"></div>

In [109]:
print(imp_num[imp_num < 10])
print('-' * 50)
print(imp_cat[imp_cat < 100])
print('-' * 50)
print(imp_bin[imp_bin < 100])

days_since_review    7.37238
dtype: float64
--------------------------------------------------
room_type_Стандарт                            3.338212
review_quarter_2                             11.216167
region_nationality_Europe                    58.525778
travelers_type_Family with older children    85.298643
travelers_type_Group                         97.835647
dtype: float64
--------------------------------------------------
standard_stay     4.282814
recent_review    70.015235
dtype: float64


Наименьшую связь с целевой переменной демонстрируют:
- 'days_since_review'
- 'review_quarter_2'
- 'region_nationality_Europe'
- 'travelers_type_Family with older children'
- 'travelers_type_Group'
- 'standard_stay' 
- 'recent_review'

Так как связь с целевой практически отсутствует (для ANOVA теста меньше 10, для хи-квадрат меньше 100) казалось, что данные признаки необходимо удалить, но как показала практика, МАРЕ тест показывает выше результат, если тут ничего не удалять. Поэтому все дальнейшие преобразования набора признаков будут выполнены на основе анализа качества самой модели.

### Нормализация

In [110]:
# Гистограммы
train_data[num_cols].hist(bins=30, figsize=(15, 10))
plt.suptitle('Гистограммы числовых признаков')
plt.tight_layout()
# plt.show
plt.close()

<div align="center">
<image src="data/image/plt_18.png"></div>

Все признаки распределены не нормально. Средние оценки отеля и процент молчаливых оценок похожи на нормальное распределения, но имеют выбросы, лучше всего для всех признаков для нормализации подходит RobustScaler.

In [111]:
# Так как нормальных распределений нет, использую RobustScaler
scaler = RobustScaler()
data[num_cols] = scaler.fit_transform(data[num_cols])
data[num_cols] = data[num_cols].astype('float32')

### После обучения модели

После некоторого количества запусков, удаляю признаки, которые менее значимы для модели:

In [112]:
features_for_drop = ['without_window', 'possibly_new_hotel', 'hi_silent_ratio', 'recent_review', 'standard_stay', 'review_2015', 'room_type_Эконом', 'room_type_Люкс', 'room_type_Стандарт', 'trip_type_other']

data.drop(features_for_drop, axis=1, inplace=True)

# Обучение модели

In [113]:
# experiment.log_table("dataset.csv", data) 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515431 entries, 0 to 515430
Data columns (total 50 columns):
 #   Column                                     Non-Null Count   Dtype   
---  ------                                     --------------   -----   
 0   additional_number_of_scoring               515431 non-null  float32 
 1   average_score                              515431 non-null  float32 
 2   review_total_negative_word_counts          515431 non-null  float32 
 3   total_number_of_reviews                    515431 non-null  float32 
 4   review_total_positive_word_counts          515431 non-null  float32 
 5   days_since_review                          515431 non-null  float32 
 6   sample                                     515431 non-null  category
 7   reviewer_score                             515431 non-null  float32 
 8   review_quarter_3_4                         515431 non-null  int8    
 9   review_quarter_2                           515431 non-null  int8    
 

In [114]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [115]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [116]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

((128935, 49), (386496, 49), (386496, 48), (309196, 48), (77300, 48))

In [117]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [118]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [119]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   48.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.4min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    1.5s finished


In [120]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
mae = metrics.mean_absolute_error(y_test, y_pred)
# experiment.log_metric("MAE", mae)
print('MAE:', mae)
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.
mape = metrics.mean_absolute_percentage_error(y_test, y_pred)
# experiment.log_metric('MAPE', mape)
print('MAPE:', mape)

MAE: 0.8775601032801563
MAPE: 0.12611455565183233


In [121]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (8,6)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
ax = feat_importances.nlargest(15).plot(kind='barh')
# Логируем график
"""
experiment.log_figure(
    figure_name="Top Important Features", figure=plt.gcf(), step=None
)
"""
# plt.show()
plt.close()

<div align="center">
<image src="data/image/plt_19.png" height="400"></div>

In [122]:
# Самые не важные признаки для модели
plt.rcParams['figure.figsize'] = (8,6)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
ax = feat_importances.nsmallest(15).plot(kind='barh')
"""
# Логируем график
experiment.log_figure(
    figure_name="Less Important Features", figure=plt.gcf(), step=None
)
"""
# plt.show()
plt.close()

<div align="center">
<image src="data/image/plt_20.png" height="400"></div>

In [123]:
# experiment.end()